# 2023 6th place solution

[リンク](https://www.kaggle.com/code/djscott1909/march-madness-2023-ds)

In [2]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf
import seaborn as sns


sys.path.append(os.path.abspath("/workspace"))
sys.path.append("C:\\Users\\81809\\Documents\\ALL_CODE\\kaggle\\March Machine Learning Mania 2025")
with initialize_config_module(version_base=None, config_module="conf"):
    cfg = compose(
        config_name="base.yaml",
        overrides=[
            "exp=exp002" #defaultexp001->exp002のオーバーライド
        ],
    )
    
    OmegaConf.resolve(cfg)
    print("Omega conf is below")
    print(OmegaConf.to_yaml(cfg))
    
"""
#こんな感じでkeyとval取得できる
print(cfg.dir.keys())
print(cfg.exp)"""

INPUT_PATH = os.path.join("..","..", cfg.dir.input_dir)
OUTPUT_PATH = cfg.dir.output_dir
SRC_PATH = cfg.dir.src_path
CONF_PATH = cfg.dir.conf_path

sys.path.append(SRC_PATH)

Omega conf is below
dir:
  input_dir: ../input
  output_dir: ../output/2025-02-22_12-20-46
  src_path: ../src
  conf_path: ../conf
model:
  epochs: 10
  lr: 0.001
exp:
  model:
    hidden_size: 100
    learning_rate: 0.1
    batch_size: 640
    num_epochs: 50



In [4]:
import os
import numpy as np
import pandas as pd
from scipy.linalg import lstsq

# データの読み込み（パスは環境に合わせて調整してください）
regular_df = pd.read_csv(os.path.join(INPUT_PATH, "MRegularSeasonCompactResults.csv"))
womens_df  = pd.read_csv(os.path.join(INPUT_PATH, "WRegularSeasonCompactResults.csv"))

# 最終的な結果を格納するDataFrameを初期化
final_SRS = pd.DataFrame()

# 関数：各試合ごとに、指定したチームIDが勝利なら1、敗北なら-1、その他なら0を返す
def transform_wl(game_data, team_id):
    # np.whereでベクトル演算
    return np.where(game_data['WTeamID'] == team_id, 1,
                    np.where(game_data['LTeamID'] == team_id, -1, 0))

# 男女共通の処理を関数化
def process_season(season_data, season_year):
    # 全チームID（勝利、敗北の両方からユニークな値）
    teams = np.unique(np.concatenate([season_data['WTeamID'].unique(), season_data['LTeamID'].unique()]))
    teams = np.sort(teams)
    
    # 各チームごとに勝敗の列ベクトルを作成（行数は試合数）
    team_columns = {}
    for team in teams:
        team_columns[str(team)] = transform_wl(season_data, team)
    # DataFrame化（列名はチームID）
    srs_df = pd.DataFrame(team_columns)
    
    # ロケーション情報の処理
    # ex_data$WLocの値： "H" -> 1, "A" -> -1, "N" -> 0
    loc_mapping = {'H': 1, 'A': -1, 'N': 0}
    srs_df.insert(0, 'loc', season_data['WLoc'].map(loc_mapping).astype(float))
    
    # 行列に変換
    srs_matrix = srs_df.values
    
    # 得点差の計算
    scorediff = (season_data['WScore'] - season_data['LScore']).values.reshape(-1, 1)
    
    # 最小二乗法で解く（scipy.linalg.lstsqはタプルを返すので、最初の要素を解とする）
    # Rのlseiは制約付き最小二乗ですが、ここでは制約なしの通常の最小二乗を用いています
    result = lstsq(srs_matrix, scorediff)[0].flatten()
    
    # 結果のDataFrame作成
    final = pd.DataFrame({
        'Team': srs_df.columns[1:],  # 先頭はlocなので除外
        'SRS': result[1:]            # 同様にlocは除外（解の順序が列順に対応）
    })
    final['Season'] = season_year
    # SRSの降順順位（SRSが高いほど順位は1に近くなる）
    final['SRS_rank'] = final['SRS'].rank(ascending=False, method='min')
    
    return final

# 男女別に処理を実施してfinal_SRSにまとめる
# まずは男子データ
for year in range(regular_df['Season'].min(), regular_df['Season'].max()+1):
    season_data = regular_df[regular_df['Season'] == year].reset_index(drop=True)
    final = process_season(season_data, year)
    final_SRS = pd.concat([final_SRS, final], ignore_index=True)
    
    # WOMENSコンポーネント：該当シーズンが存在する場合のみ処理
    if year >= womens_df['Season'].min():
        season_data_w = womens_df[womens_df['Season'] == year].reset_index(drop=True)
        final_w = process_season(season_data_w, year)
        final_SRS = pd.concat([final_SRS, final_w], ignore_index=True)

# CSVとして出力（出力先パスは環境に合わせて調整してください）
final_SRS.to_csv("M_SRS_day.csv", index=False)


In [5]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# データ読み込み（環境に合わせてパスを調整してください）
RegularSeasonCompactResults = pd.read_csv(os.path.join(INPUT_PATH, "MRegularSeasonCompactResults.csv"))
# New_modelはDayNum > 75にフィルタ
New_model = RegularSeasonCompactResults[RegularSeasonCompactResults['DayNum'] > 75].copy()

# グローバル変数（最適化後に結果を保持するため）
final_elo = None
all_games = None

def test(params):
    """
    params: [K, H, score]
    各シーズンごとにELO更新を行い、対象ゲーム(New_model)での二乗誤差平均を返す
    """
    K, H, score = params
    all_games_list = []
    final_elo_list = []
    
    # シーズンごとに処理（シーズンはRegularSeasonCompactResultsの範囲で）
    seasons = RegularSeasonCompactResults['Season'].unique()
    seasons.sort()
    
    for year in seasons:
        # 該当シーズンの全ゲーム（全試合を時系列に処理）
        season_data = RegularSeasonCompactResults[RegularSeasonCompactResults['Season'] == year].copy()
        season_data = season_data.sort_values('DayNum').reset_index(drop=True)
        
        # 結果（result）と得点差（margin）の算出
        # NumOT > 0 の場合は引き分けとみなし、result=0.5、得点差=0とする
        season_data['result'] = np.where(season_data['NumOT'] > 0, 0.5, 1.0)
        season_data['margin'] = np.where(season_data['NumOT'] > 0, 0, season_data['WScore'] - season_data['LScore'])
        # ホームロケーション：WLocが"H"なら1、"A"なら-1、"N"なら0。パラメータHを乗算
        loc_map = {'H': 1, 'A': -1, 'N': 0}
        season_data['Loc'] = season_data['WLoc'].map(loc_map).astype(float) * H

        # シーズンごとのチームレーティング（初期値1500）
        season_ratings = {}
        p_A_list = []  # 各試合の勝者予想勝率を保存
        
        # 各試合ごとにELO更新を実施
        for idx, game in season_data.iterrows():
            w_team = game['WTeamID']
            l_team = game['LTeamID']
            # 現在のレーティング（なければ1500）
            R_w = season_ratings.get(w_team, 1500.0)
            R_l = season_ratings.get(l_team, 1500.0)
            # ホームアドバンテージを勝者に反映
            R_w_adj = R_w + game['Loc']
            # 勝者の予想勝率（ロジスティック関数）
            expected = 1.0 / (1.0 + 10 ** ((R_l - R_w_adj) / 400))
            p_A_list.append(expected)
            # ゲームごとのKファクター
            k_factor = K + game['margin'] * score
            # 実際の結果（resultは0.5なら引き分け、1なら勝利）
            outcome = game['result']
            # ELO更新：勝者は outcome - expected、敗者は反対方向に更新
            new_R_w = R_w + k_factor * (outcome - expected)
            new_R_l = R_l - k_factor * (outcome - expected)
            season_ratings[w_team] = new_R_w
            season_ratings[l_team] = new_R_l
        
        # 各ゲームに勝者予想勝率を記録
        season_data['p_A'] = p_A_list
        all_games_list.append(season_data)
        
        # 最終シーズンレーティングをDataFrameにまとめる
        for team, rating in season_ratings.items():
            final_elo_list.append({'TeamID': team, 'ELO': rating, 'Season': year})
    
    all_games_df = pd.concat(all_games_list, ignore_index=True)
    final_elo_df = pd.DataFrame(final_elo_list)
    
    # 対象となるNew_modelとシーズンゲーム情報をマージ
    merge_cols = ['Season', 'DayNum', 'WTeamID', 'LTeamID']
    merged = pd.merge(all_games_df, New_model[merge_cols], on=merge_cols, how='inner')
    
    # 二乗誤差の計算：実際のresultと予想勝率p_Aの差の二乗
    merged['error'] = (merged['result'] - merged['p_A'])**2
    mse = merged['error'].mean()
    
    # グローバル変数に結果を保持（最適化後の結果確認用）
    global final_elo, all_games
    final_elo = final_elo_df
    all_games = all_games_df
    
    return mse

# 初期パラメータ [K, H, score] = [0, 0, 0]
init_params = [0, 0, 0]
# bounds: K >= 0, Hとscoreは制限なし
bounds = [(0, None), (None, None), (None, None)]

# 最適化実行
res = minimize(test, x0=init_params, bounds=bounds)
print("最適化結果:")
print(res)

# 最適化後のfinal_eloはグローバル変数として保持
final_elo_M = final_elo


最適化結果:
  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: 0.1767327883147555
        x: [ 3.603e+00  9.962e+01  4.308e+00]
      nit: 18
      jac: [-5.385e-07  1.665e-08 -4.657e-06]
     nfev: 76
     njev: 19
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>


In [6]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# ───────────────────────────────────────────
# Womensデータの読み込みと前処理
# ───────────────────────────────────────────
WRegularSeasonCompactResults = womens_df
# DayNum > 75 の試合のみ抽出
New_model_W = WRegularSeasonCompactResults[WRegularSeasonCompactResults['DayNum'] > 75].copy()

# グローバル変数：最適化後のWomens最終ELOと全ゲームデータを保持
final_elo_global = None
all_games_global = None

def test_w(params):
    """
    パラメータ: params = [K, H, score]
    各シーズンごとにELOレーティングを更新し、
    New_model_Wに含まれる試合での (実績result と予想勝率p_A) の二乗誤差平均を返す
    """
    K, H, score = params
    all_games_list = []
    final_elo_list = []
    
    # シーズンごとに処理（昇順にソート）
    seasons = np.sort(WRegularSeasonCompactResults['Season'].unique())
    
    for year in seasons:
        season_data = WRegularSeasonCompactResults[WRegularSeasonCompactResults['Season'] == year].copy()
        season_data = season_data.sort_values('DayNum').reset_index(drop=True)
        
        # 試合結果の処理: OTがあれば result=0.5（引き分けとみなす）、それ以外は1
        season_data['result'] = np.where(season_data['NumOT'] > 0, 0.5, 1.0)
        # 得点差: OTなら0、それ以外は WScore - LScore
        season_data['margin'] = np.where(season_data['NumOT'] > 0, 0, season_data['WScore'] - season_data['LScore'])
        # ホームロケーション: "H"→1, "A"→-1, "N"→0 に変換し、パラメータHを乗算
        loc_map = {'H': 1, 'A': -1, 'N': 0}
        season_data['Loc'] = season_data['WLoc'].map(loc_map).astype(float) * H

        # 各チームの初期レーティングは1500とする
        season_ratings = {}
        p_A_list = []  # 各試合での勝者の予想勝率を保持
        
        # 各試合ごとにELO更新
        for idx, game in season_data.iterrows():
            w_team = game['WTeamID']
            l_team = game['LTeamID']
            # 現在のレーティング（存在しなければ1500）
            R_w = season_ratings.get(w_team, 1500.0)
            R_l = season_ratings.get(l_team, 1500.0)
            # 勝者のレーティングにホームアドバンテージを加味
            R_w_adj = R_w + game['Loc']
            # 勝者（WTeamID）の勝率予測：ロジスティック関数による計算
            expected = 1.0 / (1.0 + 10 ** ((R_l - R_w_adj) / 400))
            p_A_list.append(expected)
            # Kファクターは K + (得点差 * score)
            k_factor = K + game['margin'] * score
            outcome = game['result']
            # ELO更新
            new_R_w = R_w + k_factor * (outcome - expected)
            new_R_l = R_l - k_factor * (outcome - expected)
            season_ratings[w_team] = new_R_w
            season_ratings[l_team] = new_R_l
        
        season_data['p_A'] = p_A_list
        all_games_list.append(season_data)
        
        # シーズン終了時の最終ELOを各チームごとに記録
        for team, rating in season_ratings.items():
            final_elo_list.append({'TeamID': team, 'ELO': rating, 'Season': year})
    
    all_games_df = pd.concat(all_games_list, ignore_index=True)
    final_elo_df = pd.DataFrame(final_elo_list)
    
    # New_model_W に含まれる試合情報とマージする（共通のキー: Season, DayNum, WTeamID, LTeamID）
    merge_cols = ['Season', 'DayNum', 'WTeamID', 'LTeamID']
    merged = pd.merge(all_games_df, New_model_W[merge_cols], on=merge_cols, how='inner')
    
    # 二乗誤差を計算し、平均値(MSE)を返す
    merged['error'] = (merged['result'] - merged['p_A'])**2
    mse = merged['error'].mean()
    
    global final_elo_global, all_games_global
    final_elo_global = final_elo_df
    all_games_global = all_games_df
    
    return mse

# 初期パラメータ [K, H, score] = [0, 0, 0]
init_params = [0, 0, 0]
# Rコードではbounds指定がなく、ここでは制約なしで最適化
res_w = minimize(test_w, x0=init_params)
print("Womens optimization result:")
print(res_w)

# 最適化後の最終ELOデータ
final_elo_W = final_elo_global

# ───────────────────────────────────────────
# 男子と女子のELO結果の結合およびCSV出力
# ───────────────────────────────────────────
# ここでは、final_elo_M（男子のELO結果）が既に計算済みであると仮定
try:
    final_elo_M
except NameError:
    # 存在しない場合は、空のDataFrameで代用（または適宜読み込み・計算してください）
    final_elo_M = pd.DataFrame()

# 男女の結果を縦方向に結合
final_elo_combined = pd.concat([final_elo_M, final_elo_W], ignore_index=True)

# CSVとして出力（パスは環境に合わせて調整してください）
final_elo_combined.to_csv("ELO_Annual_M.csv", index=False)


Womens optimization result:
  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 0.16062174902747056
        x: [ 3.614e+00  7.779e+01  5.035e+00]
      nit: 35
      jac: [ 9.406e-07  2.094e-06  8.373e-06]
 hess_inv: [[ 8.151e+02  1.828e+04  4.129e+02]
            [ 1.828e+04  4.122e+05  7.783e+03]
            [ 4.129e+02  7.783e+03  1.622e+03]]
     nfev: 144
     njev: 36


In [8]:
import numpy as np
import pandas as pd

# ───────────────────────────────────────────────
# データの読み込み
# ───────────────────────────────────────────────
m_reg = pd.read_csv(os.path.join(INPUT_PATH,"MRegularSeasonCompactResults.csv"))
ncaa = pd.read_csv(os.path.join(INPUT_PATH,"MNCAATourneyCompactResults.csv"))
sec = pd.read_csv(os.path.join(INPUT_PATH,"MSecondaryTourneyCompactResults.csv"))

# New_model：DayNum > 75 の試合のみ抽出
New_model = m_reg[m_reg['DayNum'] > 75].copy()

# 各シーズンごとの最大DayNumを求め、New_modelに新規列max_DayNumとして追加
New_model['max_DayNum'] = New_model.groupby('Season')['DayNum'].transform('max')

# ───────────────────────────────────────────────
# test関数：パラメータ (K, H, score, YEAR) に基づいてELOモデルを実行し、誤差平均（MSE）を返す
# ───────────────────────────────────────────────
def test(params):
    K, H, score, YEAR = params
    
    # 3種類の大会データを縦に結合
    # SecondaryTourneyCompactResults は、m_regと同じ列数のみ抽出
    sec_subset = sec[m_reg.columns]
    results = pd.concat([m_reg, ncaa, sec_subset], ignore_index=True)
    
    # Season, DayNumの昇順にソート
    results.sort_values(['Season', 'DayNum'], inplace=True)
    
    # 試合結果：OT試合なら result=0.5、通常は1
    results['result'] = np.where(results['NumOT'] > 0, 0.5, 1.0)
    # 得点差：OT試合なら0、それ以外は WScore - LScore
    results['margin'] = np.where(results['NumOT'] > 0, 0, results['WScore'] - results['LScore'])
    # ホームロケーション：WLoc が "H"→1, "A"→-1, "N"→0 にマッピングし、パラメータHを乗算
    results['Loc'] = results['WLoc'].map({'H': 1, 'A': -1, 'N': 0}).fillna(0) * H

    # ───────── ELO更新処理 ─────────
    # ※ 以下は dummy の処理例です。実際の elo.run の動作（adjust, regressの処理）に応じて調整してください。
    pA_list = []   # 各試合での勝者の予想勝率
    eloA_list = [] # 勝者更新後のレーティング
    eloB_list = [] # 敗者更新後のレーティング
    ratings = {}   # 各チームのレーティング（初期値は1500）

    for i, row in results.iterrows():
        w_team = row['WTeamID']
        l_team = row['LTeamID']
        # 現在のレーティング（なければ1500）
        R_w = ratings.get(w_team, 1500.0)
        R_l = ratings.get(l_team, 1500.0)
        # regression adjustment のダミー処理：
        # 実際は Season と YEAR の関係に基づく調整が必要
        R_w_adj = R_w + (row['Season'] - YEAR)
        # 勝者の予想勝率（ロジスティック関数）
        expected = 1.0 / (1.0 + 10 ** ((R_l - R_w_adj) / 400))
        pA_list.append(expected)
        # ゲームごとのKファクター
        k_factor = K + row['margin'] * score
        outcome = row['result']
        # ELO更新
        new_R_w = R_w + k_factor * (outcome - expected)
        new_R_l = R_l - k_factor * (outcome - expected)
        ratings[w_team] = new_R_w
        ratings[l_team] = new_R_l
        eloA_list.append(new_R_w)
        eloB_list.append(new_R_l)
    
    # 計算結果を新たな列として追加
    results['p.A'] = pA_list
    results['elo.A'] = eloA_list
    results['elo.B'] = eloB_list

    # results2: Rでは cbind(results, elo_results) と同等
    results2 = results.copy()

    # New_model に含まれる試合（DayNum>75）との内部結合
    merge_cols = ['Season', 'DayNum', 'WTeamID', 'LTeamID']
    results3 = pd.merge(results2, New_model[merge_cols], on=merge_cols, how='inner')
    
    # 全ゲームデータをグローバル変数に格納
    global all_games
    all_games = results3.copy()
    
    # 誤差: (実績 result と予想 p.A の差の二乗)
    results3['error'] = (results3['result'] - results3['p.A'])**2
    mse = results3['error'].mean()
    return mse

# ※実際の最適化等の前に、仮パラメータでtest関数を実行しておく
params_dummy = [20, 5, 0.1, 2021]  # 例：K=20, H=5, score=0.1, YEAR=2021
_ = test(params_dummy)

# ───────────────────────────────────────────────
# チーム毎の最終ELO値の抽出
# ───────────────────────────────────────────────
# all_games（test関数内で設定済み）のデータから、チームIDとシーズンの組み合わせを取得
teams_years = (
    all_games[['WTeamID', 'LTeamID', 'Season']]
    .drop_duplicates()
    .assign(Team=lambda df: df['WTeamID'].fillna(df['LTeamID']))
    [['Team', 'Season']]
    .drop_duplicates()
)

# 勝者リスト：WTeamID, Season, DayNum, elo.A を取得し、列名をリネーム
WTeam_list = (
    all_games[['WTeamID', 'Season', 'DayNum', 'elo.A']]
    .rename(columns={'WTeamID': 'Team', 'elo.A': 'ELO'})
)

# 敗者リスト：LTeamID, Season, DayNum, elo.B を取得し、列名をリネーム
LTeam_list = (
    all_games[['LTeamID', 'Season', 'DayNum', 'elo.B']]
    .rename(columns={'LTeamID': 'Team', 'elo.B': 'ELO'})
)

# 両者を結合してTeam_listを作成
Team_list = pd.concat([WTeam_list, LTeam_list], ignore_index=True)
Team_list.sort_values(['Season', 'DayNum', 'Team'], inplace=True)

# teams_years と内部結合し、各 (Team, Season) 毎に最後のELO値を抽出
Team_test = (
    pd.merge(Team_list, teams_years, on=['Team', 'Season'], how='inner')
    .sort_values(['Season', 'DayNum'])
    .groupby(['Team', 'Season'], as_index=False)
    .agg(Last_ELO=('ELO', 'last'))
)

Team_test_M = Team_test

# 表示（例として先頭10件）
print(Team_test_M.head(10))


   Team  Season     Last_ELO
0  1101    2014  1342.549572
1  1101    2015  1225.350495
2  1101    2016  1230.969279
3  1101    2017  1225.816870
4  1101    2018  1258.112792
5  1101    2019  1432.641712
6  1101    2020  1438.269553
7  1101    2021  1539.541648
8  1101    2022  1580.091095
9  1101    2023  1482.546551


In [9]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# ─────────────────────────────────────────────
# データ読み込み
# ─────────────────────────────────────────────
WRegularSeasonCompactResults = pd.read_csv(os.path.join(INPUT_PATH,"WRegularSeasonCompactResults.csv"))
WNCAATourneyCompactResults = pd.read_csv(os.path.join(INPUT_PATH,"WNCAATourneyCompactResults.csv"))

# New_model：DayNum > 75 の試合のみ抽出
New_model = WRegularSeasonCompactResults[WRegularSeasonCompactResults['DayNum'] > 75].copy()

# 各シーズンごとの最大DayNumをNew_modelに追加
New_model['max_DayNum'] = New_model.groupby('Season')['DayNum'].transform('max')

# ─────────────────────────────────────────────
# グローバル変数（test関数内で全試合データを保持）
# ─────────────────────────────────────────────
all_games = None

# ─────────────────────────────────────────────
# ダミーのELO更新処理を含むtest関数の定義
# パラメータ: [K, H, score, YEAR]
# ※ adjust(...), regress(...) の処理は、ここではシンプルに
#     ・ホーム効果: WLocが"H"なら1, "A"なら-1, "N"なら0 にHを乗じる
#     ・regress: 今回は Season と YEAR の関係をそのまま利用（調整が必要なら改変）
# を仮実装しています。
# ─────────────────────────────────────────────
def test(params):
    K, H, score, YEAR = params

    # ３種類の大会データを結合（ここではレギュラーとNCAAトーナメントのみ）
    results = pd.concat([WRegularSeasonCompactResults, WNCAATourneyCompactResults], ignore_index=True)
    results.sort_values(['Season', 'DayNum'], inplace=True)
    results.reset_index(drop=True, inplace=True)
    
    # 試合結果：OT試合なら result=0.5、通常は1
    results['result'] = np.where(results['NumOT'] > 0, 0.5, 1.0)
    # 得点差：OT試合なら0、それ以外は (WScore - LScore)
    results['margin'] = np.where(results['NumOT'] > 0, 0, results['WScore'] - results['LScore'])
    # ホームロケーション：WLocが"H"なら1, "A"なら-1, "N"なら0 に変換し、Hを乗算
    results['Loc'] = results['WLoc'].map({'H': 1, 'A': -1, 'N': 0}).fillna(0) * H

    # ───────── ダミーのELO更新処理 ─────────
    # 各チームの初期レーティングは1500とし、シーズンやYEARによるregress調整も
    # （今回は単純に YEAR と Season の差を加える形の調整例）
    ratings = {}   # チームIDごとの現在のレーティング
    pA_list = []   # 各試合での勝者予想勝率
    eloA_list = [] # 勝者の更新後レーティング
    eloB_list = [] # 敗者の更新後レーティング

    for idx, row in results.iterrows():
        w_team = row['WTeamID']
        l_team = row['LTeamID']
        R_w = ratings.get(w_team, 1500.0)
        R_l = ratings.get(l_team, 1500.0)
        # regress: ここでは Season と YEAR の差を単純に加算（必要に応じて調整）
        R_w_adj = R_w + (row['Season'] - YEAR)
        # 勝者の予想勝率（ロジスティック関数）
        expected = 1.0 / (1.0 + 10 ** ((R_l - R_w_adj) / 400))
        pA_list.append(expected)
        k_factor = K + row['margin'] * score
        outcome = row['result']
        new_R_w = R_w + k_factor * (outcome - expected)
        new_R_l = R_l - k_factor * (outcome - expected)
        ratings[w_team] = new_R_w
        ratings[l_team] = new_R_l
        eloA_list.append(new_R_w)
        eloB_list.append(new_R_l)
        
    # ダミーのELO更新結果をDataFrameに追加
    results['p.A'] = pA_list
    results['elo.A'] = eloA_list
    results['elo.B'] = eloB_list

    # results2: Rではcbind(results, elo_results)に相当
    results2 = results.copy()
    
    # New_modelに含まれる試合情報との内部結合（キー: Season, DayNum, WTeamID, LTeamID）
    merge_cols = ['Season', 'DayNum', 'WTeamID', 'LTeamID']
    results3 = pd.merge(results2, New_model[merge_cols], on=merge_cols, how='right')
    
    # グローバル変数に全ゲームデータを保持
    global all_games
    all_games = results3.copy()
    
    # 誤差: (実績 result と予想 p.A の差の二乗)
    results3['error'] = (results3['result'] - results3['p.A'])**2
    mse = results3['error'].mean()
    return mse

# ─────────────────────────────────────────────
# 最適化の実行
# 初期パラメータ: [K, H, score, YEAR] = [0, 0, 0, 0]
# lower: K, H, scoreは制約なし (-∞～∞)、YEARは0以上、upper: YEARは1以下（Rコードのupper指定に合わせる）
# ※ 実際はYEARの意味に合わせて適切なboundsに調整してください。
# ─────────────────────────────────────────────
init_params = [0, 0, 0, 0]
bounds = [(-np.inf, np.inf), (-np.inf, np.inf), (-np.inf, np.inf), (0, 1)]
res = minimize(test, x0=init_params, bounds=bounds)
print("最適化結果:")
print(res)

# test関数実行後の全試合データ（all_games）を利用
test_df = all_games.copy()

# ─────────────────────────────────────────────
# 各チーム・シーズンの最終ELO抽出
# ─────────────────────────────────────────────
# teams_years: WTeamID, LTeamID, Seasonのユニーク組み合わせから、チームIDを取得
teams_years = (
    test_df[['WTeamID', 'LTeamID', 'Season']]
    .drop_duplicates()
    .assign(Team=lambda df: df['WTeamID'].where(df['WTeamID'].notna(), df['LTeamID']))
    [['Team', 'Season']]
    .drop_duplicates()
)

# 勝者リスト: WTeamID, Season, DayNum, elo.A → 列名を統一して"Team"と"ELO"とする
WTeam_list = (
    test_df[['WTeamID', 'Season', 'DayNum', 'elo.A']]
    .rename(columns={'WTeamID': 'Team', 'elo.A': 'ELO'})
)

# 敗者リスト: LTeamID, Season, DayNum, elo.B
LTeam_list = (
    test_df[['LTeamID', 'Season', 'DayNum', 'elo.B']]
    .rename(columns={'LTeamID': 'Team', 'elo.B': 'ELO'})
)

# 両者を結合してTeam_listを作成
Team_list = pd.concat([WTeam_list, LTeam_list], ignore_index=True)
Team_list.sort_values(['Season', 'DayNum', 'Team'], inplace=True)

# teams_years と内部結合し、各 (Team, Season) ごとに最後のELO値を抽出
Team_test = (
    pd.merge(Team_list, teams_years, on=['Team', 'Season'], how='inner')
    .sort_values(['Season', 'DayNum'])
    .groupby(['Team', 'Season'], as_index=False)
    .agg(Last_ELO=('ELO', 'last'))
)

# この結果を Team_test_W とする
Team_test_W = Team_test.copy()

# ※ 以下は、既に計算済みの男子データ Team_test_M と結合する例です。
# もし Team_test_M が存在しない場合は、Team_test_W のみの出力となります。
try:
    Team_test_M
except NameError:
    Team_test_M = pd.DataFrame()

Team_test_total = pd.concat([Team_test_M, Team_test_W], ignore_index=True)

# CSV出力（パスは環境に合わせて調整してください）
Team_test_total.to_csv("ELO_Total_M.csv", index=False)


最適化結果:
  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: 0.009104575393844341
        x: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00]
      nit: 0
      jac: [ 0.000e+00  0.000e+00  1.735e-10 -1.561e-09]
     nfev: 5
     njev: 1
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>


In [10]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# ─────────────────────────────────────────────
# データの読み込み
# ─────────────────────────────────────────────
m_reg = pd.read_csv(os.path.join(INPUT_PATH,"MRegularSeasonCompactResults.csv"))
ncaa = pd.read_csv(os.path.join(INPUT_PATH,"MNCAATourneyCompactResults.csv"))
sec = pd.read_csv(os.path.join(INPUT_PATH,"MSecondaryTourneyCompactResults.csv"))
TeamConferences = pd.read_csv(os.path.join(INPUT_PATH,"MTeamConferences.csv"))

# ─────────────────────────────────────────────
# 統合大会データの作成
# SecondaryTourneyCompactResultsは、m_regと同じ列数のみ利用
sec_subset = sec[m_reg.columns]
results = pd.concat([m_reg, ncaa, sec_subset], ignore_index=True)
results.sort_values(['Season', 'DayNum'], inplace=True)
results.reset_index(drop=True, inplace=True)

# ─────────────────────────────────────────────
# カンファレンス情報の結合
# 左結合（WTeamID と LTeamIDそれぞれ）
results = results.merge(TeamConferences[['TeamID','Season','ConfAbbrev']], left_on=['WTeamID','Season'], right_on=['TeamID','Season'], how='left')
results.rename(columns={'ConfAbbrev': 'WConfAbbrev'}, inplace=True)
results.drop(columns=['TeamID'], inplace=True)

results = results.merge(TeamConferences[['TeamID','Season','ConfAbbrev']], left_on=['LTeamID','Season'], right_on=['TeamID','Season'], how='left')
results.rename(columns={'ConfAbbrev': 'LConfAbbrev'}, inplace=True)
results.drop(columns=['TeamID'], inplace=True)

# 絞り込み：WConfAbbrev と LConfAbbrev が異なる試合のみ残す
results = results[results['WConfAbbrev'] != results['LConfAbbrev']].copy()

# ─────────────────────────────────────────────
# New_model の作成：DayNum > 75 かつ DayNum <= max(RegularSeasonCompactResults$DayNum)
max_day = m_reg['DayNum'].max()
New_model = results[(results['DayNum'] > 75) & (results['DayNum'] <= max_day)].copy()

# ─────────────────────────────────────────────
# test関数の定義
# パラメータ: [K, H, score, YEAR]
# ※ 以下のELO更新処理はダミー実装です。
#  ・result: OT試合なら0.5、通常は1
#  ・margin: OTなら0、その他は (WScore - LScore)
#  ・group: Season*1000 + DayNum
#  ・Loc: WLocが"H"なら1, "A"なら-1, "N"なら0 に H を乗算
#  ・adjust, regress, group の処理はシンプルなダミー処理として実装
# ─────────────────────────────────────────────
def test(params):
    K, H, score, YEAR = params

    df = results.copy()
    # 結果、得点差、group, Locの計算
    df['result'] = np.where(df['NumOT'] > 0, 0.5, 1.0)
    df['margin'] = np.where(df['NumOT'] > 0, 0, df['WScore'] - df['LScore'])
    df['group'] = df['Season'] * 1000 + df['DayNum']
    df['Loc'] = df['WLoc'].map({'H': 1, 'A': -1, 'N': 0}).fillna(0) * H

    # ダミーELO更新処理
    # 各カンファレンスごとにELO更新を行う（初期値1500）
    ratings = {}  # キーはカンファレンス名
    pA_list = []   # 勝者予想勝率
    eloA_list = [] # 勝者更新後ELO
    eloB_list = [] # 敗者更新後ELO

    # 以下、試合ごとに時系列で処理
    for idx, row in df.iterrows():
        # カンファレンスをキーとして利用（adjust: カンファレンスごとに更新）
        conf_w = row['WConfAbbrev']
        conf_l = row['LConfAbbrev']
        R_w = ratings.get(conf_w, 1500.0)
        R_l = ratings.get(conf_l, 1500.0)
        # regress: 簡易的に Season と YEAR の差を加算（調整が必要なら修正）
        R_w_adj = R_w + (row['Season'] - YEAR)
        # 勝者の予想勝率（ロジスティック関数）
        expected = 1.0 / (1.0 + 10 ** ((R_l - R_w_adj) / 400))
        pA_list.append(expected)
        k_factor = K + row['margin'] * score
        outcome = row['result']
        new_R_w = R_w + k_factor * (outcome - expected)
        new_R_l = R_l - k_factor * (outcome - expected)
        ratings[conf_w] = new_R_w
        ratings[conf_l] = new_R_l
        eloA_list.append(new_R_w)
        eloB_list.append(new_R_l)

    df['p.A'] = pA_list
    df['elo.A'] = eloA_list
    df['elo.B'] = eloB_list

    # results2: ダミー処理結果のコピー
    results2 = df.copy()

    # New_modelに含まれる試合との内部結合（キー: Season, DayNum, WConfAbbrev, LConfAbbrev）
    merge_cols = ['Season', 'DayNum', 'WConfAbbrev', 'LConfAbbrev']
    results3 = pd.merge(results2, New_model[merge_cols], on=merge_cols, how='right')

    # グローバル変数に格納（デバッグ用）
    global all_games, global_results2
    all_games = results3.copy()
    global_results2 = results2.copy()

    # 誤差計算： (result - p.A)^2 の平均
    results3['error'] = (results3['result'] - results3['p.A'])**2
    mse = results3['error'].mean()
    return mse

# ─────────────────────────────────────────────
# 最適化実行
# 初期パラメータ: [K, H, score, YEAR] = [0, 0, 0, 0]
# bounds: YEARは0～1、その他は制約なし（必要に応じて調整）
# ─────────────────────────────────────────────
init_params = [0, 0, 0, 0]
bounds = [(-np.inf, np.inf), (-np.inf, np.inf), (-np.inf, np.inf), (0, 1)]
res = minimize(test, x0=init_params, bounds=bounds)
print("最適化結果:")
print(res)

# test関数実行後の全試合データ（all_games, global_results2）が利用可能
test_df = all_games.copy()
results2_df = global_results2.copy()

# ─────────────────────────────────────────────
# 各カンファレンス・シーズンごとの最終ELO抽出
# ─────────────────────────────────────────────
# teams_years: WConfAbbrev, LConfAbbrev, Seasonのユニーク組み合わせからカンファレンスを抽出
teams_years = (
    results2_df[['WConfAbbrev', 'LConfAbbrev', 'Season']]
    .drop_duplicates()
    .assign(ConfAbbrev=lambda df: df['WConfAbbrev'].where(df['WConfAbbrev'].notna(), df['LConfAbbrev']))
    [['ConfAbbrev', 'Season']]
    .drop_duplicates()
)

# 勝者リスト: WConfAbbrev, Season, DayNum, elo.A
WTeam_list = results2_df[['WConfAbbrev', 'Season', 'DayNum', 'elo.A']].copy()
WTeam_list.rename(columns={'WConfAbbrev': 'ConfAbbrev', 'elo.A': 'ELO'}, inplace=True)

# 敗者リスト: LConfAbbrev, Season, DayNum, elo.B
LTeam_list = results2_df[['LConfAbbrev', 'Season', 'DayNum', 'elo.B']].copy()
LTeam_list.rename(columns={'LConfAbbrev': 'ConfAbbrev', 'elo.B': 'ELO'}, inplace=True)

# 両者を結合してTeam_listを作成
Team_list = pd.concat([WTeam_list, LTeam_list], ignore_index=True)
Team_list.sort_values(['Season', 'DayNum', 'ConfAbbrev'], inplace=True)

# teams_years と内部結合し、各 (ConfAbbrev, Season) ごとに最後のELO値を抽出
Team_test = (
    pd.merge(Team_list, teams_years, on=['ConfAbbrev', 'Season'], how='inner')
    .sort_values(['Season', 'DayNum'])
    .groupby(['ConfAbbrev', 'Season'], as_index=False)
    .agg(Last_ELO=('ELO', 'last'))
)

Team_test_M = Team_test

# 結果の確認（例として先頭5件）
print(Team_test_M.head(5))

# CSV出力（必要に応じて出力先パスを調整）
Team_test_M.to_csv("ELO_Total_M.csv", index=False)


最適化結果:
  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: 0.00587157740898904
        x: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00]
      nit: 0
      jac: [-2.602e-10  0.000e+00  8.674e-11 -1.128e-09]
     nfev: 5
     njev: 1
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
  ConfAbbrev  Season  Last_ELO
0      a_sun    1985    1500.0
1      a_sun    1986    1500.0
2      a_sun    1987    1500.0
3      a_sun    1988    1500.0
4      a_sun    1989    1500.0


In [12]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# ─────────────────────────────────────────────
# データ読み込み
# ─────────────────────────────────────────────
w_reg = pd.read_csv(os.path.join(INPUT_PATH,"WRegularSeasonCompactResults.csv"))
w_ncaa = pd.read_csv(os.path.join(INPUT_PATH,"WNCAATourneyCompactResults.csv"))
TeamConferences = pd.read_csv(os.path.join(INPUT_PATH,"WTeamConferences.csv"))

# ─────────────────────────────────────────────
# 大会データの統合とソート
# ─────────────────────────────────────────────
results = pd.concat([w_reg, w_ncaa], ignore_index=True)
results.sort_values(['Season', 'DayNum'], inplace=True)
results.reset_index(drop=True, inplace=True)

# ─────────────────────────────────────────────
# チームカンファレンス情報の結合
# 左結合：WTeamID→WConfAbbrev
results = results.merge(TeamConferences[['TeamID', 'Season', 'ConfAbbrev']],
                        left_on=['WTeamID', 'Season'],
                        right_on=['TeamID', 'Season'],
                        how='left')
results.rename(columns={'ConfAbbrev': 'WConfAbbrev'}, inplace=True)
results.drop(columns=['TeamID'], inplace=True)

# 左結合：LTeamID→LConfAbbrev
results = results.merge(TeamConferences[['TeamID', 'Season', 'ConfAbbrev']],
                        left_on=['LTeamID', 'Season'],
                        right_on=['TeamID', 'Season'],
                        how='left')
results.rename(columns={'ConfAbbrev': 'LConfAbbrev'}, inplace=True)
results.drop(columns=['TeamID'], inplace=True)

# カンファレンスが異なる試合のみ抽出
results = results[results['WConfAbbrev'] != results['LConfAbbrev']].copy()

# ─────────────────────────────────────────────
# New_modelの作成：DayNum > 75 かつ DayNum <= 最大日付（w_regの最大値）
max_day = w_reg['DayNum'].max()
New_model = results[(results['DayNum'] > 75) & (results['DayNum'] <= max_day)].copy()

# ─────────────────────────────────────────────
# ダミーのELO更新処理を含むtest関数の定義
# パラメータ: [K, H, score, YEAR]
# ※ adjust, regress, group の処理は簡略化しています。
#   ・result: NumOT > 0なら0.5、そうでなければ1
#   ・margin: NumOT > 0なら0、そうでなければ (WScore - LScore)
#   ・group: Season*1000 + DayNum
#   ・Loc: WLocが"H"なら1、"A"なら-1、"N"なら0 に H を乗算
# ─────────────────────────────────────────────
def test(params):
    K, H, score, YEAR = params
    df = results.copy()
    
    df['result'] = np.where(df['NumOT'] > 0, 0.5, 1.0)
    df['margin'] = np.where(df['NumOT'] > 0, 0, df['WScore'] - df['LScore'])
    df['group'] = df['Season'] * 1000 + df['DayNum']
    df['Loc'] = df['WLoc'].map({'H': 1, 'A': -1, 'N': 0}).fillna(0) * H

    # ダミーELO更新処理（カンファレンスごとの更新）
    # 各カンファレンスの初期レーティングは1500
    ratings = {}  # キーはカンファレンス名
    pA_list = []   # 各試合での勝者予想勝率
    eloA_list = [] # 勝者更新後のELO
    eloB_list = [] # 敗者更新後のELO

    for idx, row in df.iterrows():
        conf_w = row['WConfAbbrev']
        conf_l = row['LConfAbbrev']
        R_w = ratings.get(conf_w, 1500.0)
        R_l = ratings.get(conf_l, 1500.0)
        # regressのダミー処理：Season と YEAR の差を加算
        R_w_adj = R_w + (row['Season'] - YEAR)
        expected = 1.0 / (1.0 + 10 ** ((R_l - R_w_adj) / 400))
        pA_list.append(expected)
        k_factor = K + row['margin'] * score
        outcome = row['result']
        new_R_w = R_w + k_factor * (outcome - expected)
        new_R_l = R_l - k_factor * (outcome - expected)
        ratings[conf_w] = new_R_w
        ratings[conf_l] = new_R_l
        eloA_list.append(new_R_w)
        eloB_list.append(new_R_l)
    
    df['p.A'] = pA_list
    df['elo.A'] = eloA_list
    df['elo.B'] = eloB_list

    # results2は更新後のdfのコピー
    global results2
    results2 = df.copy()

    # New_modelに含まれる試合と内部結合（キー: Season, DayNum, WConfAbbrev, LConfAbbrev）
    merge_cols = ['Season', 'DayNum', 'WConfAbbrev', 'LConfAbbrev']
    results3 = pd.merge(results2, New_model[merge_cols], on=merge_cols, how='right')
    
    global all_games
    all_games = results3.copy()
    
    # MSEの計算： (result - p.A)^2 の平均
    results3['error'] = (results3['result'] - results3['p.A'])**2
    mse = results3['error'].mean()
    return mse

# ─────────────────────────────────────────────
# 最適化実行
# 初期パラメータ: [K, H, score, YEAR] = [0, 0, 0, 0]
# YEARは0～1の範囲（必要に応じて調整）
# ─────────────────────────────────────────────
init_params = [0, 0, 0, 0]
bounds = [(-np.inf, np.inf), (-np.inf, np.inf), (-np.inf, np.inf), (0, 1)]
res = minimize(test, x0=init_params, bounds=bounds)
print("最適化結果:")
print(res)

# test関数実行後の全試合データ（all_games, results2）が利用可能
test_df = all_games.copy()
results2_df = results2.copy()

# ─────────────────────────────────────────────
# 各カンファレンス・シーズンごとの最終ELO抽出
# ─────────────────────────────────────────────
# teams_years: WConfAbbrev, LConfAbbrev, Seasonのユニーク組み合わせからカンファレンスを抽出
teams_years = (
    results2_df[['WConfAbbrev', 'LConfAbbrev', 'Season']]
    .drop_duplicates()
    .assign(ConfAbbrev=lambda df: df['WConfAbbrev'].where(df['WConfAbbrev'].notna(), df['LConfAbbrev']))
    [['ConfAbbrev', 'Season']]
    .drop_duplicates()
)

# 勝者リスト: WConfAbbrev, Season, DayNum, elo.A → 列名を統一して"ConfAbbrev"と"ELO"に変更
WTeam_list = results2_df[['WConfAbbrev', 'Season', 'DayNum', 'elo.A']].copy()
WTeam_list.rename(columns={'WConfAbbrev': 'ConfAbbrev', 'elo.A': 'ELO'}, inplace=True)

# 敗者リスト: LConfAbbrev, Season, DayNum, elo.B
LTeam_list = results2_df[['LConfAbbrev', 'Season', 'DayNum', 'elo.B']].copy()
LTeam_list.rename(columns={'LConfAbbrev': 'ConfAbbrev', 'elo.B': 'ELO'}, inplace=True)

# 両者を結合してTeam_listを作成
Team_list = pd.concat([WTeam_list, LTeam_list], ignore_index=True)
Team_list.sort_values(['Season', 'DayNum', 'ConfAbbrev'], inplace=True)

# teams_yearsと内部結合し、各 (ConfAbbrev, Season) ごとに最新のELO値を抽出
Team_test = (
    pd.merge(Team_list, teams_years, on=['ConfAbbrev', 'Season'], how='inner')
    .sort_values(['Season', 'DayNum'])
    .groupby(['ConfAbbrev', 'Season'], as_index=False)
    .agg(Last_ELO=('ELO', 'last'))
)

Team_test_W = Team_test.copy()

# ─────────────────────────────────────────────
# CSV出力
# ※ 男子データTeam_test_Mは既に算出済みとし、ここでは男女の結果を別々に出力する例
# ─────────────────────────────────────────────

Team_test_W.to_csv("ELO_CONF_W.csv", index=False)
Team_test_M.to_csv("ELO_CONF_M.csv", index=False)


最適化結果:
  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: 0.0058591575875841665
        x: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00]
      nit: 0
      jac: [-2.602e-10  0.000e+00  8.674e-11 -1.214e-09]
     nfev: 5
     njev: 1
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>


In [14]:
import pandas as pd
import numpy as np

# データ読み込み
df = pd.read_csv(os.path.join(INPUT_PATH,"MRegularSeasonCompactResults.csv"))

# 結果を格納するリスト（シーズン毎の結果）
final_PF_list = []

# シーズンの範囲でループ
for year in range(int(df['Season'].min()), int(df['Season'].max()) + 1):
    # 該当シーズンのデータ抽出
    ex_data = df[df['Season'] == year].copy()
    
    # --- sql_w: 勝った側の集計 ---
    sql_w = (ex_data[['WTeamID', 'Season', 'WScore', 'LScore']]
             .rename(columns={'WTeamID': 'Team'})
             .groupby(['Team', 'Season'], as_index=False)
             .agg(WPTS_for=('WScore', 'sum'),
                  WPTS_against=('LScore', 'sum'),
                  WGames=('WScore', 'count')))
    
    # --- sql_l: 負けた側の集計 ---
    sql_l = (ex_data[['LTeamID', 'Season', 'LScore', 'WScore']]
             .rename(columns={'LTeamID': 'Team'})
             .groupby(['Team', 'Season'], as_index=False)
             .agg(LPTS_for=('LScore', 'sum'),
                  LPTS_against=('WScore', 'sum'),
                  LGames=('LScore', 'count')))
    
    # --- 両者のフルジョイン（outer merge） ---
    sql = pd.merge(sql_w, sql_l, on=['Team', 'Season'], how='outer')
    sql.fillna(0, inplace=True)
    
    # 勝率・平均得点の計算
    total_games = sql['WGames'] + sql['LGames']
    sql['Win_perc'] = sql['WGames'] / total_games
    sql['PTS_for'] = (sql['WPTS_for'] + sql['LPTS_for']) / total_games
    sql['PTS_against'] = (sql['WPTS_against'] + sql['LPTS_against']) / total_games
    sql['Season'] = year  # シーズンを明示
    
    # --- SOS の計算 ---
    # ex_data に対し、勝者側のWin_perc（＝W_Win_perc）を結合
    SOS = pd.merge(ex_data, sql[['Team', 'Season', 'Win_perc']],
                   left_on=['WTeamID', 'Season'], right_on=['Team', 'Season'], how='left')
    SOS.rename(columns={'Win_perc': 'W_Win_perc'}, inplace=True)
    SOS.drop(columns=['Team'], inplace=True)
    
    # 同様に、敗者側のWin_perc（＝L_Win_perc）を結合
    SOS = pd.merge(SOS, sql[['Team', 'Season', 'Win_perc']],
                   left_on=['LTeamID', 'Season'], right_on=['Team', 'Season'], how='left')
    SOS.rename(columns={'Win_perc': 'L_Win_perc'}, inplace=True)
    SOS.drop(columns=['Team'], inplace=True)
    
    # SOSW: 勝者側のデータ（チーム＝WTeamIDから、相手の勝率：L_Win_perc）を集計
    SOSW = (SOS[['WTeamID', 'L_Win_perc']]
            .rename(columns={'WTeamID': 'Team'})
            .groupby('Team', as_index=False)
            .agg(W_Win_perc=('L_Win_perc', 'sum'),
                 Wgames=('L_Win_perc', 'count')))
    
    # SOSL: 敗者側のデータ（チーム＝LTeamIDから、相手の勝率：W_Win_perc）を集計
    SOSL = (SOS[['LTeamID', 'W_Win_perc']]
            .rename(columns={'LTeamID': 'Team'})
            .groupby('Team', as_index=False)
            .agg(L_Win_perc=('W_Win_perc', 'sum'),
                 Lgames=('W_Win_perc', 'count')))
    
    # 両者のマージ（外部結合）
    SOSF = pd.merge(SOSW, SOSL, on='Team', how='outer')
    SOSF.fillna(0, inplace=True)
    
    # 各チームのSOS＝(W_Win_perc + L_Win_perc) / (Wgames + Lgames)
    SOSF['SOS'] = (SOSF['W_Win_perc'] + SOSF['L_Win_perc']) / (SOSF['Wgames'] + SOSF['Lgames'])
    # SOSF（調整値）：各チームのSOSからシーズン内平均を引く
    mean_SOS = SOSF['SOS'].mean()
    SOSF['SOSF'] = SOSF['SOS'] - mean_SOS
    
    # sql に対して、SOSを左結合（結合後は不要な中間集計列は除外）
    sql = pd.merge(sql, SOSF[['Team', 'SOS']], on='Team', how='left')
    sql = sql[['Team', 'Win_perc', 'PTS_for', 'PTS_against', 'SOS', 'Season']]
    
    # --- SOOS の計算（対戦相手のSOS値を用いる） ---
    # ex_data に対して、勝者側のSOSを結合（キー：WTeamID）
    SOOS = pd.merge(ex_data, sql[['Team', 'Season', 'SOS']],
                    left_on=['WTeamID', 'Season'], right_on=['Team', 'Season'], how='left')
    SOOS.rename(columns={'SOS': 'W_SOS'}, inplace=True)
    SOOS.drop(columns=['Team'], inplace=True)
    
    # 同様に、敗者側のSOSを結合（キー：LTeamID）
    SOOS = pd.merge(SOOS, sql[['Team', 'Season', 'SOS']],
                    left_on=['LTeamID', 'Season'], right_on=['Team', 'Season'], how='left')
    SOOS.rename(columns={'SOS': 'L_SOS'}, inplace=True)
    SOOS.drop(columns=['Team'], inplace=True)
    
    # SOOSW: 勝者側のデータ（Team＝WTeamIDから、相手のSOS：L_SOS）を集計
    SOOSW = (SOOS[['WTeamID', 'L_SOS']]
             .rename(columns={'WTeamID': 'Team'})
             .groupby('Team', as_index=False)
             .agg(W_SOS=('L_SOS', 'sum'),
                  Wgames=('L_SOS', 'count')))
    
    # SOOSL: 敗者側のデータ（Team＝LTeamIDから、相手のSOS：W_SOS）を集計
    SOOSL = (SOOS[['LTeamID', 'W_SOS']]
             .rename(columns={'LTeamID': 'Team'})
             .groupby('Team', as_index=False)
             .agg(L_SOS=('W_SOS', 'sum'),
                  Lgames=('W_SOS', 'count')))
    
    # 両者をマージ
    SOOSF = pd.merge(SOOSW, SOOSL, on='Team', how='outer')
    SOOSF.fillna(0, inplace=True)
    
    # 各チームのSOOS＝(W_SOS + L_SOS) / (Wgames + Lgames)
    SOOSF['SOOS'] = (SOOSF['W_SOS'] + SOOSF['L_SOS']) / (SOOSF['Wgames'] + SOOSF['Lgames'])
    mean_SOOS = SOOSF['SOOS'].mean()
    SOOSF['SOOSF'] = SOOSF['SOOS'] - mean_SOOS
    
    # sql に対して、SOOSを左結合
    sql = pd.merge(sql, SOOSF[['Team', 'SOOS']], on='Team', how='left')
    # SOS_final = (SOS*2 + SOOS) / 3
    sql['SOS_final'] = (sql['SOS'] * 2 + sql['SOOS']) / 3
    # SOOS列は不要なので削除
    sql.drop(columns=['SOOS'], inplace=True)
    
    # シーズンごとの結果（sql）をリストに追加
    final_PF_list.append(sql)

# すべてのシーズンの結果を結合
final_PF_M = pd.concat(final_PF_list, ignore_index=True)

# 必要に応じて、CSV出力（例）
# final_PF_M.to_csv("/kaggle/working/final_PF_M.csv", index=False)

print(final_PF_M.head())


   Team  Win_perc    PTS_for  PTS_against       SOS  Season  SOS_final
0  1102  0.208333  63.083333    68.875000  0.500130    1985   0.497775
1  1103  0.391304  61.043478    64.086957  0.474244    1985   0.473840
2  1104  0.700000  68.500000    60.700000  0.526081    1985   0.531555
3  1106  0.416667  71.625000    75.416667  0.462600    1985   0.462814
4  1108  0.760000  83.000000    75.040000  0.439307    1985   0.452978


In [17]:
import pandas as pd
import numpy as np

# ------------------------------
# 1. データ読み込み
# ------------------------------
w_reg = pd.read_csv(os.path.join(INPUT_PATH,"WRegularSeasonCompactResults.csv"))
# ※ 男子データは既に計算済みとして、例えば以下のように読み込み（必要に応じて調整）
final_PF_M = pd.read_csv("Final_Stats_M.csv")  # 例：男子データ

# ------------------------------
# 2. シーズンごとの集計処理
# ------------------------------
final_PF_list = []

for year in range(int(w_reg['Season'].min()), int(w_reg['Season'].max()) + 1):
    # 該当シーズンのデータ
    ex_data = w_reg[w_reg['Season'] == year].copy()
    
    # --- sql_w: 勝った側の集計 ---
    sql_w = (ex_data[['WTeamID', 'Season', 'WScore', 'LScore']]
             .rename(columns={'WTeamID': 'Team'})
             .groupby(['Team', 'Season'], as_index=False)
             .agg(WPTS_for = ('WScore', 'sum'),
                  WPTS_against = ('LScore', 'sum'),
                  WGames = ('WScore', 'count')))
    
    # --- sql_l: 負けた側の集計 ---
    sql_l = (ex_data[['LTeamID', 'Season', 'LScore', 'WScore']]
             .rename(columns={'LTeamID': 'Team'})
             .groupby(['Team', 'Season'], as_index=False)
             .agg(LPTS_for = ('LScore', 'sum'),
                  LPTS_against = ('WScore', 'sum'),
                  LGames = ('LScore', 'count')))
    
    # --- 両者のフルジョイン ---
    sql = pd.merge(sql_w, sql_l, on=['Team', 'Season'], how='outer')
    sql.fillna(0, inplace=True)
    
    # 勝率、１試合平均得点（得点・失点）の算出
    total_games = sql['WGames'] + sql['LGames']
    sql['Win_perc'] = sql['WGames'] / total_games
    sql['PTS_for'] = (sql['WPTS_for'] + sql['LPTS_for']) / total_games
    sql['PTS_against'] = (sql['WPTS_against'] + sql['LPTS_against']) / total_games
    sql['Season'] = year  # 念のためシーズン情報を更新
    
    # --- SOSの計算 ---
    # 勝者側の勝率を結合（WTeamID → Win_percをW_Win_percとして取得）
    SOS = pd.merge(ex_data, sql[['Team', 'Season', 'Win_perc']],
                   left_on=['WTeamID','Season'], right_on=['Team','Season'], how='left')
    SOS.rename(columns={'Win_perc': 'W_Win_perc'}, inplace=True)
    SOS.drop(columns=['Team'], inplace=True)
    # 敗者側の勝率を結合（LTeamID → Win_percをL_Win_percとして取得）
    SOS = pd.merge(SOS, sql[['Team', 'Season', 'Win_perc']],
                   left_on=['LTeamID','Season'], right_on=['Team','Season'], how='left')
    SOS.rename(columns={'Win_perc': 'L_Win_perc'}, inplace=True)
    SOS.drop(columns=['Team'], inplace=True)
    
    # SOSW: 勝者側（WTeamIDから、相手の勝率：L_Win_perc）の集計
    SOSW = (SOS[['WTeamID', 'L_Win_perc']]
            .rename(columns={'WTeamID': 'Team'})
            .groupby('Team', as_index=False)
            .agg(W_Win_perc = ('L_Win_perc', 'sum'),
                 Wgames = ('L_Win_perc', 'count')))
    
    # SOSL: 敗者側（LTeamIDから、相手の勝率：W_Win_perc）の集計
    SOSL = (SOS[['LTeamID', 'W_Win_perc']]
            .rename(columns={'LTeamID': 'Team'})
            .groupby('Team', as_index=False)
            .agg(L_Win_perc = ('W_Win_perc', 'sum'),
                 Lgames = ('W_Win_perc', 'count')))
    
    # フルジョインしてSOSFを作成
    SOSF = pd.merge(SOSW, SOSL, on='Team', how='outer')
    SOSF.fillna(0, inplace=True)
    
    # 各チームのSOS＝(W_Win_perc + L_Win_perc) / (Wgames + Lgames)
    SOSF['SOS'] = (SOSF['W_Win_perc'])


FileNotFoundError: [Errno 2] No such file or directory: 'Final_Stats_M.csv'

In [ ]:
import pandas as pd
import numpy as np

# --- 1. データ読み込み ---
# RegularSeasonCompactResults と SRS_day のCSVファイルを読み込み
results = pd.read_csv(os.apeh.join(INPUT_PATH,"MRegularSeasonCompactResults.csv", sep=","))
SRS_day = pd.read_csv("M_SRS_day.csv", sep=",")

# --- 2. 試合ごとのチームIDの設定 ---
# Team1: 小さい方のチームID、Team2: 大きい方のチームID
results['Team1'] = np.where(results['WTeamID'] < results['LTeamID'], results['WTeamID'], results['LTeamID'])
results['Team2'] = np.where(results['WTeamID'] > results['LTeamID'], results['WTeamID'], results['LTeamID'])

# --- 3. SRS_day の前処理 ---
# "loc" 以外の行を残し、Team列を数値型に変換
SRS_day = SRS_day[SRS_day['Team'] != "loc"].copy()
SRS_day['TeamID'] = pd.to_numeric(SRS_day['Team'])
SRS_day = SRS_day[['TeamID', 'Season', 'SRS', 'SRS_rank']]

# --- 4. SRS_day の結合 ---
# Team1 についてSRS_dayを左結合
results = results.merge(SRS_day, how='left', left_on=['Team1', 'Season'], right_on=['TeamID', 'Season'])
results.rename(columns={'SRS': 'SRS_T1', 'SRS_rank': 'SRS_rank_T1'}, inplace=True)
results.drop(columns=['TeamID'], inplace=True)
# Team2 についても同様
results = results.merge(SRS_day, how='left', left_on=['Team2', 'Season'], right_on=['TeamID', 'Season'])
results.rename(columns={'SRS': 'SRS_T2', 'SRS_rank': 'SRS_rank_T2'}, inplace=True)
results.drop(columns=['TeamID'], inplace=True)

# --- 5. loc_T1 の設定 ---
# loc_T1は、WLocが "N" の場合は "N"、それ以外は、Team1がWTeamIDならWLoc、そうでなければ逆（WLocが "H" なら "A"、そうでなければ "H"）
results['loc_T1'] = np.where(results['WLoc'] == "N", "N",
                     np.where(results['Team1'] == results['WTeamID'],
                              results['WLoc'],
                              np.where(results['WLoc'] == "H", "A", "H")))

# --- 6. quant_T1, quant_T2 の設定 ---
# quant_T1：対戦相手（Team2）のSRS_rank_T2 とloc_T1により条件分岐
results['quant_T1'] = np.where((results['loc_T1'] == "H") & (results['SRS_rank_T2'] <= 30), 1,
                        np.where((results['loc_T1'] == "N") & (results['SRS_rank_T2'] <= 50), 1,
                        np.where((results['loc_T1'] == "A") & (results['SRS_rank_T2'] <= 75), 1,
                        np.where((results['loc_T1'] == "H") & (results['SRS_rank_T2'] <= 75), 2,
                        np.where((results['loc_T1'] == "N") & (results['SRS_rank_T2'] <= 100), 2,
                        np.where((results['loc_T1'] == "A") & (results['SRS_rank_T2'] <= 135), 2, 3))))))

# quant_T2：対戦相手（Team1）のSRS_rank_T1 とloc_T1により条件分岐（逆条件）
results['quant_T2'] = np.where((results['loc_T1'] == "A") & (results['SRS_rank_T1'] <= 30), 1,
                        np.where((results['loc_T1'] == "N") & (results['SRS_rank_T1'] <= 50), 1,
                        np.where((results['loc_T1'] == "H") & (results['SRS_rank_T1'] <= 75), 1,
                        np.where((results['loc_T1'] == "A") & (results['SRS_rank_T1'] <= 75), 2,
                        np.where((results['loc_T1'] == "N") & (results['SRS_rank_T1'] <= 100), 2,
                        np.where((results['loc_T1'] == "H") & (results['SRS_rank_T1'] <= 135), 2, 3))))))

# --- 7. 試合結果の設定 ---
# results$result = 1 if Team1 == WTeamID、それ以外は0
results['result'] = np.where(results['WTeamID'] < results['LTeamID'], 1, 0)

# --- 8. 集計：Team1側の集計 ---
results_T1 = (results[['Season', 'Team1', 'quant_T1', 'result']]
              .rename(columns={'Team1': 'Team', 'quant_T1': 'quant'}))
results_T1 = results_T1.groupby(['Season','Team','quant'], as_index=False).agg(Wins_T1=('result', 'sum'),
                                                                              games_T1=('result', 'count'))

# --- 9. 集計：Team2側の集計 ---
results_T2 = (results[['Season', 'Team2', 'quant_T2', 'result']]
              .rename(columns={'Team2': 'Team', 'quant_T2': 'quant'}))
# ここでは、Team2側は敗北側なので勝ち数は (1 - result)
results_T2['result'] = 1 - results_T2['result']
results_T2 = results_T2.groupby(['Season','Team','quant'], as_index=False).agg(Wins_T2=('result', 'sum'),
                                                                              games_T2=('result', 'count'))

# --- 10. 両者の結合・合算 ---
results_final = pd.merge(results_T1, results_T2, on=['Season','Team','quant'], how='outer').fillna(0)
results_final['Wins'] = results_final['Wins_T1'] + results_final['Wins_T2']
results_final['games'] = results_final['games_T1'] + results_final['games_T2']

# --- 11. クォンタイル別の抽出 ---
quant1 = results_final[results_final['quant'] == 1][['Season','Team','Wins','games']].rename(columns={'Wins': 'Wins_quant1', 'games': 'games_quant1'})
quant2 = results_final[results_final['quant'] == 2][['Season','Team','Wins','games']].rename(columns={'Wins': 'Wins_quant2', 'games': 'games_quant2'})
quant3 = results_final[results_final['quant'] == 3][['Season','Team','Wins','games']].rename(columns={'Wins': 'Wins_quant3', 'games': 'games_quant3'})

final_quant = quant1.merge(quant2, on=['Season','Team'], how='outer') \
                    .merge(quant3, on=['Season','Team'], how='outer').fillna(0)

# --- 12. 中立試合の集計 ---
neutral = results[results['loc_T1'] == "N"][['Season', 'Team1', 'Team2', 'result']]
neutral_T1 = neutral.groupby(['Season','Team1'], as_index=False).agg(Wins_Neutral_T1=('result', 'sum'),
                                                                      games_Neutral_T1=('result', 'count'))
neutral_T2 = neutral.groupby(['Season','Team2'], as_index=False).agg(Wins_Neutral_T2=('result', lambda x: (1 - x).sum()),
                                                                      games_Neutral_T2=('result', 'count'))
neutral_final = pd.merge(neutral_T1.rename(columns={'Team1':'Team'}),
                         neutral_T2.rename(columns={'Team2':'Team'}),
                         on=['Season','Team'], how='outer').fillna(0)
neutral_final['Wins_Neutral'] = neutral_final['Wins_Neutral_T1'] + neutral_final['Wins_Neutral_T2']
neutral_final['games_Neutral'] = neutral_final['games_Neutral_T1'] + neutral_final['games_Neutral_T2']
neutral_final = neutral_final[['Season','Team','Wins_Neutral','games_Neutral']]

# --- 13. アウェイ試合の集計 ---
# away_t1: loc_T1=="A" の試合（Team1側）
away_t1 = results[results['loc_T1'] == "A"][['Season','Team1','Team2','result']]
away_t1 = away_t1.groupby(['Season','Team1'], as_index=False).agg(Wins_Away_T1=('result', 'sum'),
                                                                 games_Away_T1=('result', 'count'))
# away_t2: loc_T1=="H" の試合（Team1側）※ RコードではTeam1側も集計
away_t2 = results[results['loc_T1'] == "H"][['Season','Team1','Team2','result']]
away_t2 = away_t2.groupby(['Season','Team1'], as_index=False).agg(Wins_Away_T2=('result', 'sum'),
                                                                 games_Away_T2=('result', 'count'))
away = pd.merge(away_t1.rename(columns={'Team1':'Team'}),
                away_t2.rename(columns={'Team1':'Team'}),
                on=['Season','Team'], how='outer').fillna(0)
away['Wins_Away'] = away['Wins_Away_T1'] + away['Wins_Away_T2']
away['games_Away'] = away['games_Away_T1'] + away['games_Away_T2']
away = away[['Season','Team','Wins_Away','games_Away']]

# --- 14. final_quant に中立・アウェイ成績を結合 ---
final_quant = final_quant.merge(neutral_final, on=['Season','Team'], how='outer') \
                         .merge(away, on=['Season','Team'], how='outer').fillna(0)

# --- 15. 指標の算出 ---
final_quant['win_perc_quant1'] = final_quant['Wins_quant1'] / final_quant['games_quant1']
final_quant['even_quant1'] = final_quant['Wins_quant1'] - (final_quant['games_quant1'] / 2)

final_quant['win_perc_quant2'] = (final_quant['Wins_quant1'] + final_quant['Wins_quant2']) / (final_quant['games_quant1'] + final_quant['games_quant2'])
final_quant['even_quant2'] = (final_quant['Wins_quant1'] + final_quant['Wins_quant2']) - ((final_quant['games_quant1'] / 2) + (final_quant['games_quant2'] / 2))

final_quant['win_perc_neutral'] = final_quant['Wins_Neutral'] / final_quant['games_Neutral']
final_quant['even_neutral'] = final_quant['Wins_Neutral'] - (final_quant['games_Neutral'] / 2)

final_quant['win_perc_away'] = final_quant['Wins_Away'] / final_quant['games_Away']
final_quant['even_away'] = final_quant['Wins_Away'] - (final_quant['games_Away'] / 2)

final_quant['win_perc_nothome'] = (final_quant['Wins_Away'] + final_quant['Wins_Neutral']) / (final_quant['games_Away'] + final_quant['games_Neutral'])
final_quant['even_nothome'] = (final_quant['Wins_Away'] + final_quant['Wins_Neutral']) - ((final_quant['games_Away'] / 2) + (final_quant['games_Neutral'] / 2))

final_quant = final_quant.fillna(0)

# --- 16. 最終結果（女子） ---
final_quant_M = final_quant.copy()  # 今回は女子データ用としてfinal_quant_Mに格納

# ここで必要に応じて、男子データ（final_quant_M）と結合するなどして出力
# 例：男子データと女子データを結合してCSV出力
# final_total = pd.concat([final_quant_M_men, final_quant_M], ignore_index=True)
final_quant_M.to_csv("Final_Stats.csv", index=False)

print(final_quant_M.head())


In [ ]:
import pandas as pd
import numpy as np

# 1. データ読み込み
results = pd.read_csv(os.apeh.join(INPUT_PATH,"MRegularSeasonCompactResults.csv", sep=","))
SRS_day = pd.read_csv("M_SRS_day.csv", sep=",")

# ※ 男子データ（既に算出済み）を読み込み（または変数に代入済みの場合）
final_quant_M = pd.read_csv("final_quant_men.csv")  # 適宜パスを調整

# 2. チームIDの決定
results['Team1'] = np.where(results['WTeamID'] < results['LTeamID'], results['WTeamID'], results['LTeamID'])
results['Team2'] = np.where(results['WTeamID'] > results['LTeamID'], results['WTeamID'], results['LTeamID'])

# 3. SRS_day の前処理
SRS_day = SRS_day[SRS_day['Team'] != "loc"].copy()
SRS_day['TeamID'] = pd.to_numeric(SRS_day['Team'])
SRS_day = SRS_day[['TeamID', 'Season', 'SRS', 'SRS_rank']]

# 4. SRS_day の結合（Team1）
results = results.merge(SRS_day, how='left', left_on=['Team1', 'Season'], right_on=['TeamID', 'Season'])
results.rename(columns={'SRS': 'SRS_T1', 'SRS_rank': 'SRS_rank_T1'}, inplace=True)
results.drop(columns=['TeamID'], inplace=True)

# 5. SRS_day の結合（Team2）
results = results.merge(SRS_day, how='left', left_on=['Team2', 'Season'], right_on=['TeamID', 'Season'])
results.rename(columns={'SRS': 'SRS_T2', 'SRS_rank': 'SRS_rank_T2'}, inplace=True)
results.drop(columns=['TeamID'], inplace=True)

# 6. loc_T1 の設定
# if WLoc=="N" → "N"
# else: if Team1 == WTeamID then use WLoc; else if WLoc=="H" then "A", otherwise "H"
results['loc_T1'] = np.where(results['WLoc'] == "N", "N",
                     np.where(results['Team1'] == results['WTeamID'],
                              results['WLoc'],
                              np.where(results['WLoc'] == "H", "A", "H")))

# 7. quant_T1, quant_T2 の設定（対戦相手のSRS_rankとloc_T1により判定）
results['quant_T1'] = np.where((results['loc_T1'] == "H") & (results['SRS_rank_T2'] <= 30), 1,
                        np.where((results['loc_T1'] == "N") & (results['SRS_rank_T2'] <= 50), 1,
                        np.where((results['loc_T1'] == "A") & (results['SRS_rank_T2'] <= 75), 1,
                        np.where((results['loc_T1'] == "H") & (results['SRS_rank_T2'] <= 75), 2,
                        np.where((results['loc_T1'] == "N") & (results['SRS_rank_T2'] <= 100), 2, 
                        np.where((results['loc_T1'] == "A") & (results['SRS_rank_T2'] <= 135), 2, 3))))))
results['quant_T2'] = np.where((results['loc_T1'] == "A") & (results['SRS_rank_T1'] <= 30), 1,
                        np.where((results['loc_T1'] == "N") & (results['SRS_rank_T1'] <= 50), 1,
                        np.where((results['loc_T1'] == "H") & (results['SRS_rank_T1'] <= 75), 1,
                        np.where((results['loc_T1'] == "A") & (results['SRS_rank_T1'] <= 75), 2,
                        np.where((results['loc_T1'] == "N") & (results['SRS_rank_T1'] <= 100), 2, 
                        np.where((results['loc_T1'] == "H") & (results['SRS_rank_T1'] <= 135), 2, 3))))))

# 8. 試合結果の設定
# result = 1 if Team1 == WTeamID (WTeamID < LTeamIDの場合に1)、それ以外は0
results['result'] = np.where(results['WTeamID'] < results['LTeamID'], 1, 0)

# 9. 集計：Team1側
results_T1 = (results[['Season', 'Team1', 'quant_T1', 'result']]
              .rename(columns={'Team1': 'Team', 'quant_T1': 'quant'}))
results_T1 = results_T1.groupby(['Season','Team','quant'], as_index=False).agg(Wins_T1=('result', 'sum'),
                                                                              games_T1=('result', 'count'))

# 10. 集計：Team2側
results_T2 = (results[['Season', 'Team2', 'quant_T2', 'result']]
              .rename(columns={'Team2': 'Team', 'quant_T2': 'quant'}))
# ここは敗者側の結果なので勝ち数は 1 - result
results_T2['result'] = 1 - results_T2['result']
results_T2 = results_T2.groupby(['Season','Team','quant'], as_index=False).agg(Wins_T2=('result', 'sum'),
                                                                              games_T2=('result', 'count'))

# 11. 両者の結合
results_final = pd.merge(results_T1, results_T2, on=['Season','Team','quant'], how='outer').fillna(0)
results_final['Wins'] = results_final['Wins_T1'] + results_final['Wins_T2']
results_final['games'] = results_final['games_T1'] + results_final['games_T2']

# 12. クォンタイル別に抽出
quant1 = results_final[results_final['quant'] == 1][['Season','Team','Wins','games']].rename(columns={'Wins': 'Wins_quant1', 'games': 'games_quant1'})
quant2 = results_final[results_final['quant'] == 2][['Season','Team','Wins','games']].rename(columns={'Wins': 'Wins_quant2', 'games': 'games_quant2'})
quant3 = results_final[results_final['quant'] == 3][['Season','Team','Wins','games']].rename(columns={'Wins': 'Wins_quant3', 'games': 'games_quant3'})

final_quant = quant1.merge(quant2, on=['Season','Team'], how='outer') \
                     .merge(quant3, on=['Season','Team'], how='outer').fillna(0)

# 13. 中立試合の集計（loc_T1 == "N"）
neutral = results[results['loc_T1'] == "N"][['Season', 'Team1', 'Team2', 'result']]
neutral_T1 = neutral.groupby(['Season', 'Team1'], as_index=False).agg(Wins_Neutral_T1=('result', 'sum'),
                                                                        games_Neutral_T1=('result', 'count'))
neutral_T2 = neutral.groupby(['Season', 'Team2'], as_index=False).agg(Wins_Neutral_T2=('result', lambda x: (1 - x).sum()),
                                                                        games_Neutral_T2=('result', 'count'))
neutral_final = pd.merge(neutral_T1.rename(columns={'Team1':'Team'}),
                         neutral_T2.rename(columns={'Team2':'Team'}),
                         on=['Season','Team'], how='outer').fillna(0)
neutral_final['Wins_Neutral'] = neutral_final['Wins_Neutral_T1'] + neutral_final['Wins_Neutral_T2']
neutral_final['games_Neutral'] = neutral_final['games_Neutral_T1'] + neutral_final['games_Neutral_T2']
neutral_final = neutral_final[['Season','Team','Wins_Neutral','games_Neutral']]

# 14. アウェイ試合の集計
# away_t1: loc_T1=="A" の場合（Team1側）
away_t1 = results[results['loc_T1'] == "A"][['Season','Team1','Team2','result']]
away_t1 = away_t1.groupby(['Season','Team1'], as_index=False).agg(Wins_Away_T1=('result', 'sum'),
                                                                  games_Away_T1=('result', 'count'))
# away_t2: loc_T1=="H" の場合（Team1側）
away_t2 = results[results['loc_T1'] == "H"][['Season','Team1','Team2','result']]
away_t2 = away_t2.groupby(['Season','Team1'], as_index=False).agg(Wins_Away_T2=('result', 'sum'),
                                                                  games_Away_T2=('result', 'count'))
away = pd.merge(away_t1.rename(columns={'Team1':'Team'}),
                away_t2.rename(columns={'Team1':'Team'}),
                on=['Season','Team'], how='outer').fillna(0)
away['Wins_Away'] = away['Wins_Away_T1'] + away['Wins_Away_T2']
away['games_Away'] = away['games_Away_T1'] + away['games_Away_T2']
away = away[['Season','Team','Wins_Away','games_Away']]

# 15. final_quant に中立・アウェイ成績を結合
final_quant = final_quant.merge(neutral_final, on=['Season','Team'], how='outer') \
                         .merge(away, on=['Season','Team'], how='outer').fillna(0)

# 16. 指標算出
final_quant['win_perc_quant1'] = final_quant['Wins_quant1'] / final_quant['games_quant1']
final_quant['even_quant1'] = final_quant['Wins_quant1'] - (final_quant['games_quant1'] / 2)

final_quant['win_perc_quant2'] = (final_quant['Wins_quant1'] + final_quant['Wins_quant2']) / (final_quant['games_quant1'] + final_quant['games_quant2'])
final_quant['even_quant2'] = (final_quant['Wins_quant1'] + final_quant['Wins_quant2']) - ((final_quant['games_quant1'] / 2) + (final_quant['games_quant2'] / 2))

final_quant['win_perc_neutral'] = final_quant['Wins_Neutral'] / final_quant['games_Neutral']
final_quant['even_neutral'] = final_quant['Wins_Neutral'] - (final_quant['games_Neutral'] / 2)

final_quant['win_perc_away'] = final_quant['Wins_Away'] / final_quant['games_Away']
final_quant['even_away'] = final_quant['Wins_Away'] - (final_quant['games_Away'] / 2)

final_quant['win_perc_nothome'] = (final_quant['Wins_Away'] + final_quant['Wins_Neutral']) / (final_quant['games_Away'] + final_quant['games_Neutral'])
final_quant['even_nothome'] = (final_quant['Wins_Away'] + final_quant['Wins_Neutral']) - ((final_quant['games_Away'] / 2) + (final_quant['games_Neutral'] / 2))

final_quant = final_quant.fillna(0)
final_quant_W = final_quant.copy()

# 17. 男子データと女子データの結合、CSV出力
quant_combined = pd.concat([final_quant_M, final_quant_W], ignore_index=True)
quant_combined.to_csv("final_quant.csv", index=False)

print(quant_combined.head())


In [ ]:
import pandas as pd
import numpy as np

# --- 1. CSVの読み込み ---
ncaatourney = pd.read_csv(os.apeh.join(INPUT_PATH,"MNCAATourneyCompactResults.csv", sep=","))
teamconf = pd.read_csv(os.apeh.join(INPUT_PATH,"MTeamConferences.csv", sep=","))
submission = pd.read_csv(os.apeh.join(INPUT_PATH,"SampleSubmission2023.csv", sep=","))
ncaatourneyseeds = pd.read_csv(os.apeh.join(INPUT_PATH,"MNCAATourneySeeds.csv", sep=","))

elo_annual = pd.read_csv("ELO_Annual_M.csv", sep=",")
elo_conf = pd.read_csv("ELO_CONF_M.csv", sep=",")
elo_total = pd.read_csv("ELO_Total_M.csv", sep=",")
srs_day = pd.read_csv("M_SRS_day.csv", sep=",")
final_stats = pd.read_csv("Final_Stats.csv", sep=",")
quant_combined = pd.read_csv("/final_quant.csv", sep=",")

# --- 2. NCAATourneyCompactResults を results に代入し、Team1, Team2 を決定 ---
results = ncaatourney.copy()
results['Team1'] = np.where(results['WTeamID'] < results['LTeamID'],
                            results['WTeamID'], results['LTeamID'])
results['Team2'] = np.where(results['WTeamID'] > results['LTeamID'],
                            results['WTeamID'], results['LTeamID'])

# --- 3. 各種結合 ---
# ELO_Annual_M 結合（Team1）
results = results.merge(elo_annual[['TeamID','Season','ELO']], how='left',
                        left_on=['Team1','Season'], right_on=['TeamID','Season'])
results.rename(columns={'ELO': 'ELO_T1'}, inplace=True)
results.drop(columns=['TeamID'], inplace=True)
# ELO_Annual_M 結合（Team2）
results = results.merge(elo_annual[['TeamID','Season','ELO']], how='left',
                        left_on=['Team2','Season'], right_on=['TeamID','Season'])
results.rename(columns={'ELO': 'ELO_T2'}, inplace=True)
results.drop(columns=['TeamID'], inplace=True)

# TeamConferences 結合（Team1）
results = results.merge(teamconf[['TeamID','Season','ConfAbbrev']], how='left',
                        left_on=['Team1','Season'], right_on=['TeamID','Season'])
results.rename(columns={'ConfAbbrev': 'ConfAbbrev_T1'}, inplace=True)
results.drop(columns=['TeamID'], inplace=True)
# TeamConferences 結合（Team2）
results = results.merge(teamconf[['TeamID','Season','ConfAbbrev']], how='left',
                        left_on=['Team2','Season'], right_on=['TeamID','Season'])
results.rename(columns={'ConfAbbrev': 'ConfAbbrev_T2'}, inplace=True)
results.drop(columns=['TeamID'], inplace=True)

# ELO_CONF 結合（Team1のカンファレンス）
results = results.merge(elo_conf[['ConfAbbrev','Season','Last_ELO']], how='left',
                        left_on=['ConfAbbrev_T1','Season'], right_on=['ConfAbbrev','Season'])
results.rename(columns={'Last_ELO': 'CONF_ELO_T1'}, inplace=True)
results.drop(columns=['ConfAbbrev'], inplace=True)
# ELO_CONF 結合（Team2のカンファレンス）
results = results.merge(elo_conf[['ConfAbbrev','Season','Last_ELO']], how='left',
                        left_on=['ConfAbbrev_T2','Season'], right_on=['ConfAbbrev','Season'])
results.rename(columns={'Last_ELO': 'CONF_ELO_T2'}, inplace=True)
results.drop(columns=['ConfAbbrev'], inplace=True)

# ELO_Total 結合（Team1）
results = results.merge(elo_total[['Team','Season','Last_ELO']], how='left',
                        left_on=['Team1','Season'], right_on=['Team','Season'])
results.rename(columns={'Last_ELO': 'ELO_ALL_T1'}, inplace=True)
results.drop(columns=['Team'], inplace=True)
# ELO_Total 結合（Team2）
results = results.merge(elo_total[['Team','Season','Last_ELO']], how='left',
                        left_on=['Team2','Season'], right_on=['Team','Season'])
results.rename(columns={'Last_ELO': 'ELO_ALL_T2'}, inplace=True)
results.drop(columns=['Team'], inplace=True)

# SRS_day の前処理
srs_day = srs_day[srs_day['Team'] != "loc"].copy()
srs_day['TeamID'] = pd.to_numeric(srs_day['Team'])
srs_day = srs_day[['TeamID','Season','SRS','SRS_rank']]

# SRS_day 結合（Team1）
results = results.merge(srs_day, how='left',
                        left_on=['Team1','Season'], right_on=['TeamID','Season'])
results.rename(columns={'SRS': 'SRS_T1', 'SRS_rank': 'SRS_rank_T1'}, inplace=True)
results.drop(columns=['TeamID'], inplace=True)
# SRS_day 結合（Team2）
results = results.merge(srs_day, how='left',
                        left_on=['Team2','Season'], right_on=['TeamID','Season'])
results.rename(columns={'SRS': 'SRS_T2', 'SRS_rank': 'SRS_rank_T2'}, inplace=True)
results.drop(columns=['TeamID'], inplace=True)

# final_stats 結合（Team1）
results = results.merge(final_stats, how='left',
                        left_on=['Team1','Season'], right_on=['Team','Season'])
results.rename(columns={'Win_perc': 'Win_perc_T1', 'PTS_for': 'PTS_for_T1',
                        'PTS_against': 'PTS_against_T1', 'SOS': 'SOS_T1',
                        'SOS_final': 'SOS_final_T1'}, inplace=True)
results.drop(columns=['Team'], inplace=True)
# final_stats 結合（Team2）
results = results.merge(final_stats, how='left',
                        left_on=['Team2','Season'], right_on=['Team','Season'])
results.rename(columns={'Win_perc': 'Win_perc_T2', 'PTS_for': 'PTS_for_T2',
                        'PTS_against': 'PTS_against_T2', 'SOS': 'SOS_T2',
                        'SOS_final': 'SOS_final_T2'}, inplace=True)
results.drop(columns=['Team'], inplace=True)

# NCAATourneySeeds 結合（Team1）
results = results.merge(ncaatourneyseeds, how='left',
                        left_on=['Team1','Season'], right_on=['TeamID','Season'])
results.rename(columns={'Seed': 'Seed_T1'}, inplace=True)
results.drop(columns=['TeamID'], inplace=True)
# NCAATourneySeeds 結合（Team2）
results = results.merge(ncaatourneyseeds, how='left',
                        left_on=['Team2','Season'], right_on=['TeamID','Season'])
results.rename(columns={'Seed': 'Seed_T2'}, inplace=True)
results.drop(columns=['TeamID'], inplace=True)

# Seed 数値化
results['SeedNum_T1'] = results['Seed_T1'].str[1:3].astype(float)
results['SeedNum_T2'] = results['Seed_T2'].str[1:3].astype(float)

# --- Quant 結合 ---
# Team1側の quant 結果
results = results.merge(quant_combined, how='left',
                        left_on=['Team1','Season'], right_on=['Team','Season'])
results.rename(columns={
    'Wins_quant1': 'Wins_quant1_T1',
    'win_perc_quant1': 'win_perc_quant1_T1',
    'even_quant1': 'even_quant1_T1',
    'Wins_quant2': 'Wins_quant2_T1',
    'win_perc_quant2': 'win_perc_quant2_T1',
    'even_quant2': 'even_quant2_T1',
    'Wins_Neutral': 'Wins_Neutral_T1',
    'win_perc_neutral': 'win_perc_neutral_T1',
    'even_neutral': 'even_neutral_T1',
    'Wins_Away': 'Wins_Away_T1',
    'win_perc_away': 'win_perc_away_T1',
    'even_away': 'even_away_T1',
    'win_perc_nothome': 'win_perc_nothome_T1',
    'even_nothome': 'even_nothome_T1'
}, inplace=True)
results.drop(columns=['Team'], inplace=True)

# Team2側の quant 結果
results = results.merge(quant_combined, how='left',
                        left_on=['Team2','Season'], right_on=['Team','Season'])
results.rename(columns={
    'Wins_quant1': 'Wins_quant1_T2',
    'win_perc_quant1': 'win_perc_quant1_T2',
    'even_quant1': 'even_quant1_T2',
    'Wins_quant2': 'Wins_quant2_T2',
    'win_perc_quant2': 'win_perc_quant2_T2',
    'even_quant2': 'even_quant2_T2',
    'Wins_Neutral': 'Wins_Neutral_T2',
    'win_perc_neutral': 'win_perc_neutral_T2',
    'even_neutral': 'even_neutral_T2',
    'Wins_Away': 'Wins_Away_T2',
    'win_perc_away': 'win_perc_away_T2',
    'even_away': 'even_away_T2',
    'win_perc_nothome': 'win_perc_nothome_T2',
    'even_nothome': 'even_nothome_T2'
}, inplace=True)
results.drop(columns=['Team'], inplace=True)

# --- 結果の設定 ---
results['result'] = np.where(results['WTeamID'] < results['LTeamID'], 1, 0)
results['Score1'] = np.where(results['WTeamID'] < results['LTeamID'], results['WScore'], results['LScore'])
results['Score2'] = np.where(results['WTeamID'] > results['LTeamID'], results['WScore'], results['LScore'])
results['Seed_len1'] = results['Seed_T1'].str[:3]
results['Seed_len2'] = results['Seed_T2'].str[:3]
results['margin'] = np.where(results['NumOT'] > 0, 0, results['Score1'] - results['Score2'])
results['Loc'] = np.where(results['WLoc'] == "H", 1, np.where(results['WLoc'] == "A", -1, 0))

# --- 差分計算 ---
results['ELO'] = results['ELO_T1'] - results['ELO_T2']
results['CONF_ELO'] = results['CONF_ELO_T1'] - results['CONF_ELO_T2']
results['ELO_ALL'] = results['ELO_ALL_T1'] - results['ELO_ALL_T2']
results['SRS'] = results['SRS_T1'] - results['SRS_T2']
results['SRS_rank'] = results['SRS_rank_T1'] - results['SRS_rank_T2']
results['Win_perc'] = results['Win_perc_T1'] - results['Win_perc_T2']
results['PTS_for'] = results['PTS_for_T1'] - results['PTS_for_T2']
results['PTS_against'] = results['PTS_against_T1'] - results['PTS_against_T2']
results['SOS'] = results['SOS_T1'] - results['SOS_T2']
results['SOS_final'] = results['SOS_final_T1'] - results['SOS_final_T2']
results['SeedNum'] = results['SeedNum_T1'] - results['SeedNum_T2']

results['Wins_quant1'] = results['Wins_quant1_T1'] - results['Wins_quant1_T2']
results['win_perc_quant1'] = results['win_perc_quant1_T1'] - results['win_perc_quant1_T2']
results['even_quant1'] = results['even_quant1_T1'] - results['even_quant1_T2']
results['Wins_quant2'] = results['Wins_quant2_T1'] - results['Wins_quant2_T2']
results['win_perc_quant2'] = results['win_perc_quant2_T1'] - results['win_perc_quant2_T2']
results['even_quant2'] = results['even_quant2_T1'] - results['even_quant2_T2']
results['Wins_Neutral'] = results['Wins_Neutral_T1'] - results['Wins_Neutral_T2']
results['win_perc_neutral'] = results['win_perc_neutral_T1'] - results['win_perc_neutral_T2']
results['even_neutral'] = results['even_neutral_T1'] - results['even_neutral_T2']
results['Wins_Away'] = results['Wins_Away_T1'] - results['Wins_Away_T2']
results['win_perc_away'] = results['win_perc_away_T1'] - results['win_perc_away_T2']
results['even_away'] = results['even_away_T1'] - results['even_away_T2']
results['win_perc_nothome'] = results['win_perc_nothome_T1'] - results['win_perc_nothome_T2']
results['even_nothome'] = results['even_nothome_T1'] - results['even_nothome_T2']

# --- 追加処理：play in game の除外 ---
results_temp = results[results['Seed_len1'] != results['Seed_len2']].copy()

results_model = results_temp.copy()
results_OOT = results_temp[results_temp['Season'] >= 2017].copy()

print("results_model 行数:", results_model.shape[0])
print("results_OOT 行数:", results_OOT.shape[0])


In [ ]:
import pandas as pd
import numpy as np

# --- 1. データ読み込み ---
Sub = pd.read_csv(os.apeh.join(INPUT_PATH,"SampleSubmission2023.csv", sep=","))
elo_annual = pd.read_csv("ELO_Annual_M.csv", sep=",")
teamconf = pd.read_csv(os.apeh.join(INPUT_PATH,"MTeamConferences.csv", sep=","))
elo_conf = pd.read_csv("ELO_CONF_M.csv", sep=",")
elo_total = pd.read_csv("ELO_Total_M.csv", sep=",")
srs_day = pd.read_csv("M_SRS_day.csv", sep=",")
final_stats = pd.read_csv("Final_Stats.csv", sep=",")
ncaatourneyseeds = pd.read_csv(os.apeh.join(INPUT_PATH,"MNCAATourneySeeds.csv", sep=","))
quant_combined = pd.read_csv("final_quant.csv", sep=",")

# --- 2. ID から Season, Team1, Team2 の抽出 ---
# R: Sub$Season <- as.numeric(substr(Sub$ID,1,4))
#     Sub$Team1 <- as.numeric(substr(Sub$ID,6,9))
#     Sub$Team2 <- as.numeric(substr(Sub$ID,11,14))
Sub['Season'] = Sub['ID'].str.slice(0,4).astype(int)
Sub['Team1'] = Sub['ID'].str.slice(5,9).astype(int)
Sub['Team2'] = Sub['ID'].str.slice(10,14).astype(int)

# --- 3. Team1 の上限フィルタ（例: Team1 <= 3100） ---
Sub = Sub[Sub['Team1'] <= 3100].copy()

# --- 4. ELO_Annual_M の結合 ---
Sub = Sub.merge(elo_annual[['TeamID','Season','ELO']], how='left',
                left_on=['Team1','Season'], right_on=['TeamID','Season'])
Sub.rename(columns={'ELO': 'ELO_T1'}, inplace=True)
Sub.drop(columns=['TeamID'], inplace=True)
Sub = Sub.merge(elo_annual[['TeamID','Season','ELO']], how='left',
                left_on=['Team2','Season'], right_on=['TeamID','Season'])
Sub.rename(columns={'ELO': 'ELO_T2'}, inplace=True)
Sub.drop(columns=['TeamID'], inplace=True)

# --- 5. TeamConferences の結合 ---
Sub = Sub.merge(teamconf[['TeamID','Season','ConfAbbrev']], how='left',
                left_on=['Team1','Season'], right_on=['TeamID','Season'])
Sub.rename(columns={'ConfAbbrev': 'ConfAbbrev_T1'}, inplace=True)
Sub.drop(columns=['TeamID'], inplace=True)
Sub = Sub.merge(teamconf[['TeamID','Season','ConfAbbrev']], how='left',
                left_on=['Team2','Season'], right_on=['TeamID','Season'])
Sub.rename(columns={'ConfAbbrev': 'ConfAbbrev_T2'}, inplace=True)
Sub.drop(columns=['TeamID'], inplace=True)

# --- 6. ELO_CONF の結合 ---
Sub = Sub.merge(elo_conf[['ConfAbbrev','Season','Last_ELO']], how='left',
                left_on=['ConfAbbrev_T1','Season'], right_on=['ConfAbbrev','Season'])
Sub.rename(columns={'Last_ELO': 'CONF_ELO_T1'}, inplace=True)
Sub.drop(columns=['ConfAbbrev'], inplace=True)
Sub = Sub.merge(elo_conf[['ConfAbbrev','Season','Last_ELO']], how='left',
                left_on=['ConfAbbrev_T2','Season'], right_on=['ConfAbbrev','Season'])
Sub.rename(columns={'Last_ELO': 'CONF_ELO_T2'}, inplace=True)
Sub.drop(columns=['ConfAbbrev'], inplace=True)

# --- 7. ELO_Total の結合 ---
Sub = Sub.merge(elo_total[['Team','Season','Last_ELO']], how='left',
                left_on=['Team1','Season'], right_on=['Team','Season'])
Sub.rename(columns={'Last_ELO': 'ELO_ALL_T1'}, inplace=True)
Sub.drop(columns=['Team'], inplace=True)
Sub = Sub.merge(elo_total[['Team','Season','Last_ELO']], how='left',
                left_on=['Team2','Season'], right_on=['Team','Season'])
Sub.rename(columns={'Last_ELO': 'ELO_ALL_T2'}, inplace=True)
Sub.drop(columns=['Team'], inplace=True)

# --- 8. SRS_day の結合 ---
# SRS_day: Team != "loc" かつ TeamID を数値変換
srs_day = srs_day[srs_day['Team'] != "loc"].copy()
srs_day['TeamID'] = pd.to_numeric(srs_day['Team'])
srs_day = srs_day[['TeamID','Season','SRS','SRS_rank']]
Sub = Sub.merge(srs_day, how='left',
                left_on=['Team1','Season'], right_on=['TeamID','Season'])
Sub.rename(columns={'SRS': 'SRS_T1', 'SRS_rank': 'SRS_rank_T1'}, inplace=True)
Sub.drop(columns=['TeamID'], inplace=True)
Sub = Sub.merge(srs_day, how='left',
                left_on=['Team2','Season'], right_on=['TeamID','Season'])
Sub.rename(columns={'SRS': 'SRS_T2', 'SRS_rank': 'SRS_rank_T2'}, inplace=True)
Sub.drop(columns=['TeamID'], inplace=True)

# --- 9. final_stats の結合 ---
Sub = Sub.merge(final_stats, how='left',
                left_on=['Team1','Season'], right_on=['Team','Season'])
Sub.rename(columns={'Win_perc': 'Win_perc_T1', 'PTS_for': 'PTS_for_T1',
                    'PTS_against': 'PTS_against_T1', 'SOS': 'SOS_T1',
                    'SOS_final': 'SOS_final_T1'}, inplace=True)
Sub.drop(columns=['Team'], inplace=True)
Sub = Sub.merge(final_stats, how='left',
                left_on=['Team2','Season'], right_on=['Team','Season'])
Sub.rename(columns={'Win_perc': 'Win_perc_T2', 'PTS_for': 'PTS_for_T2',
                    'PTS_against': 'PTS_against_T2', 'SOS': 'SOS_T2',
                    'SOS_final': 'SOS_final_T2'}, inplace=True)
Sub.drop(columns=['Team'], inplace=True)

# --- 10. NCAATourneySeeds の結合 ---
Sub = Sub.merge(ncaatourneyseeds, how='left',
                left_on=['Team1','Season'], right_on=['TeamID','Season'])
Sub.rename(columns={'Seed': 'Seed_T1'}, inplace=True)
Sub.drop(columns=['TeamID'], inplace=True)
Sub = Sub.merge(ncaatourneyseeds, how='left',
                left_on=['Team2','Season'], right_on=['TeamID','Season'])
Sub.rename(columns={'Seed': 'Seed_T2'}, inplace=True)
Sub.drop(columns=['TeamID'], inplace=True)

# --- 11. Seed 番号の抽出 ---
Sub['SeedNum_T1'] = Sub['Seed_T1'].str.slice(1,3).astype(float)
Sub['SeedNum_T2'] = Sub['Seed_T2'].str.slice(1,3).astype(float)

# --- 12. Quant 指標の結合 ---
# Team1 側の quant 結果
Sub = Sub.merge(quant_combined, how='left',
                left_on=['Team1','Season'], right_on=['Team','Season'])
Sub.rename(columns={
    'Wins_quant1': 'Wins_quant1_T1',
    'win_perc_quant1': 'win_perc_quant1_T1',
    'even_quant1': 'even_quant1_T1',
    'Wins_quant2': 'Wins_quant2_T1',
    'win_perc_quant2': 'win_perc_quant2_T1',
    'even_quant2': 'even_quant2_T1',
    'Wins_Neutral': 'Wins_Neutral_T1',
    'win_perc_neutral': 'win_perc_neutral_T1',
    'even_neutral': 'even_neutral_T1',
    'Wins_Away': 'Wins_Away_T1',
    'win_perc_away': 'win_perc_away_T1',
    'even_away': 'even_away_T1',
    'win_perc_nothome': 'win_perc_nothome_T1',
    'even_nothome': 'even_nothome_T1'
}, inplace=True)
Sub.drop(columns=['Team'], inplace=True)
# Team2 側の quant 結果
Sub = Sub.merge(quant_combined, how='left',
                left_on=['Team2','Season'], right_on=['Team','Season'])
Sub.rename(columns={
    'Wins_quant1': 'Wins_quant1_T2',
    'win_perc_quant1': 'win_perc_quant1_T2',
    'even_quant1': 'even_quant1_T2',
    'Wins_quant2': 'Wins_quant2_T2',
    'win_perc_quant2': 'win_perc_quant2_T2',
    'even_quant2': 'even_quant2_T2',
    'Wins_Neutral': 'Wins_Neutral_T2',
    'win_perc_neutral': 'win_perc_neutral_T2',
    'even_neutral': 'even_neutral_T2',
    'Wins_Away': 'Wins_Away_T2',
    'win_perc_away': 'win_perc_away_T2',
    'even_away': 'even_away_T2',
    'win_perc_nothome': 'win_perc_nothome_T2',
    'even_nothome': 'even_nothome_T2'
}, inplace=True)
Sub.drop(columns=['Team'], inplace=True)

# --- 13. 差分計算 ---
Sub['ELO'] = Sub['ELO_T1'] - Sub['ELO_T2']
Sub['CONF_ELO'] = Sub['CONF_ELO_T1'] - Sub['CONF_ELO_T2']
Sub['ELO_ALL'] = Sub['ELO_ALL_T1'] - Sub['ELO_ALL_T2']
Sub['SRS'] = Sub['SRS_T1'] - Sub['SRS_T2']
Sub['SRS_rank'] = Sub['SRS_rank_T1'] - Sub['SRS_rank_T2']
Sub['Win_perc'] = Sub['Win_perc_T1'] - Sub['Win_perc_T2']
Sub['PTS_for'] = Sub['PTS_for_T1'] - Sub['PTS_for_T2']
Sub['PTS_against'] = Sub['PTS_against_T1'] - Sub['PTS_against_T2']
Sub['SOS'] = Sub['SOS_T1'] - Sub['SOS_T2']
Sub['SOS_final'] = Sub['SOS_final_T1'] - Sub['SOS_final_T2']
Sub['SeedNum'] = Sub['SeedNum_T1'] - Sub['SeedNum_T2']

Sub['Wins_quant1'] = Sub['Wins_quant1_T1'] - Sub['Wins_quant1_T2']
Sub['win_perc_quant1'] = Sub['win_perc_quant1_T1'] - Sub['win_perc_quant1_T2']
Sub['even_quant1'] = Sub['even_quant1_T1'] - Sub['even_quant1_T2']
Sub['Wins_quant2'] = Sub['Wins_quant2_T1'] - Sub['Wins_quant2_T2']
Sub['win_perc_quant2'] = Sub['win_perc_quant2_T1'] - Sub['win_perc_quant2_T2']
Sub['even_quant2'] = Sub['even_quant2_T1'] - Sub['even_quant2_T2']
Sub['Wins_Neutral'] = Sub['Wins_Neutral_T1'] - Sub['Wins_Neutral_T2']
Sub['win_perc_neutral'] = Sub['win_perc_neutral_T1'] - Sub['win_perc_neutral_T2']
Sub['even_neutral'] = Sub['even_neutral_T1'] - Sub['even_neutral_T2']
Sub['Wins_Away'] = Sub['Wins_Away_T1'] - Sub['Wins_Away_T2']
Sub['win_perc_away'] = Sub['win_perc_away_T1'] - Sub['win_perc_away_T2']
Sub['even_away'] = Sub['even_away_T1'] - Sub['even_away_T2']
Sub['win_perc_nothome'] = Sub['win_perc_nothome_T1'] - Sub['win_perc_nothome_T2']
Sub['even_nothome'] = Sub['even_nothome_T1'] - Sub['even_nothome_T2']

# --- 14. 結果の出力 ---
# ここまでで Sub に各種指標の差分が算出されています
# 必要に応じて以降でさらにモデリング用に加工できます

# 以下は、最終的な Sub の内容確認例
print(Sub.head())

# 例: Sub を CSV として保存
Sub.to_csv("Submission_Features.csv", index=False)


In [ ]:
import numpy as np
import pandas as pd
import scorecardpy as sc

def variable_transform(var):
    global results_model, results_OOT, Sub
    
    # --- 学習データ (results_model) の処理 ---
    # 1. 絶対値と変数の符号に応じた目的変数 TEMP_result を作成
    results_model["TEMP_ABS"] = results_model[var].abs()
    results_model["TEMP_result"] = np.where(results_model[var] >= 0,
                                            results_model["result"],
                                            1 - results_model["result"])
    # 2. scorecardpy を用いて TEMP_ABS の binning を実施
    #    ※ y = "TEMP_result", x = "TEMP_ABS" で binning を行い、binning 結果の辞書を取得
    binning_dict = sc.woebin(results_model, y="TEMP_result", x=["TEMP_ABS"], no_cores=1)
    # ※ binning_dict は { "TEMP_ABS": DataFrame } となる
    Bin_proposed = binning_dict["TEMP_ABS"]
    
    # 3. IV 値の表示
    print(var)
    print("IV =", Bin_proposed["total_iv"].iloc[0] if "total_iv" in Bin_proposed.columns else "IV not computed")
    
    # 4. 学習データに対して binning 変数を作成（新規列名は例えば var+"_binned"）
    #    ※ scorecardpy.woebin_ply で binning 変数を作成
    binned_col = var + "_binned"
    results_model = sc.woebin_ply(results_model, binning_dict, x=["TEMP_ABS"], var_skip=[], keep="all")
    # ※ 上記で生成される新規列名は "TEMP_ABS_binned"
    # リネームして var+"_binned" に変更
    results_model.rename(columns={"TEMP_ABS_binned": binned_col}, inplace=True)
    
    # 5. ここでは、R と同様に先頭2文字（bin 番号）を数値として抽出
    results_model["binning"] = results_model[binned_col].astype(str).str[:2].astype(int)
    
    # 6. binning 用の IV テーブルを準備
    merge_temp = Bin_proposed.copy().reset_index(drop=True)
    merge_temp["cut"] = np.arange(1, len(merge_temp) + 1)
    # WoE の Inf 対策（R の場合と同様の対処）
    # ※ここでは各行の WoE 値が無限大の場合に、全体の Good/Bad 数から調整する例
    # ※実際はデータに合わせて処理を調整してください
    merge_temp["WoE"] = merge_temp["WoE"].replace(np.inf,
                        np.log((merge_temp["CntGood"] / merge_temp["CntGood"].sum()) / (1 / merge_temp["CntBad"].sum())))
    merge_temp["WoE"] = merge_temp["WoE"].replace(-np.inf,
                        np.log((1 / merge_temp["CntGood"].sum()) / (merge_temp["CntBad"] / merge_temp["CntBad"].sum())))
    
    # 7. merge_temp の WoE を results_model にマージし、最終的な WoE 列 (var+"_WoE") を作成
    results_model = results_model.merge(merge_temp[["cut", "WoE"]], left_on="binning", right_on="cut", how="left")
    results_model[var + "_WoE"] = np.where(results_model[var] >= 0,
                                             results_model["WoE"],
                                             -1 * results_model["WoE"])
    results_model.drop(columns=["WoE", "cut"], inplace=True)
    
    # --- 検証データ (results_OOT) の処理 ---
    results_OOT["TEMP_ABS"] = results_OOT[var].abs()
    # 利用する binning_dict は学習データで作成したものと同じ
    results_OOT = sc.woebin_ply(results_OOT, binning_dict, x=["TEMP_ABS"], var_skip=[], keep="all")
    # 上記で生成される列名は "TEMP_ABS_binned" → リネーム
    results_OOT.rename(columns={"TEMP_ABS_binned": binned_col}, inplace=True)
    results_OOT["binning"] = results_OOT[binned_col].astype(str).str[:2].astype(int)
    results_OOT = results_OOT.merge(merge_temp[["cut", "WoE"]], left_on="binning", right_on="cut", how="left")
    results_OOT[var + "_WoE"] = np.where(results_OOT[var] >= 0,
                                         results_OOT["WoE"],
                                         -1 * results_OOT["WoE"])
    results_OOT.drop(columns=["WoE", "cut"], inplace=True)
    
    # --- 提出用データ (Sub) の処理 ---
    Sub["TEMP_ABS"] = Sub[var].abs()
    # ここでは、直接 findInterval 相当の処理として np.digitize を使用
    # なお、Bin_proposed には "bands" 列（各区間の境界値）がある前提
    if "bands" in Bin_proposed.columns:
        # np.digitize では、右側境界をどう扱うか注意
        Sub[binned_col] = np.digitize(Sub["TEMP_ABS"], bins=Bin_proposed["bands"].values, right=False)
    else:
        # bands 列がない場合は、スコアカードパッケージと同様の出力が得られていると仮定
        Sub[binned_col] = sc.woebin_ply(Sub, binning_dict, x=["TEMP_ABS"], var_skip=[], keep="all")[binned_col]
    Sub["binning"] = Sub[binned_col].astype(str).str[:2].astype(int)
    Sub = Sub.merge(merge_temp[["cut", "WoE"]], left_on="binning", right_on="cut", how="left")
    Sub[var + "_WoE"] = np.where(Sub[var] >= 0,
                                 Sub["WoE"],
                                 -1 * Sub["WoE"])
    Sub.drop(columns=["WoE", "cut"], inplace=True)
    
    # 各データセットをグローバル更新
    results_model.drop(columns=["TEMP_ABS", "TEMP_result", "binning"], inplace=True)
    results_OOT.drop(columns=["TEMP_ABS", "binning"], inplace=True)
    Sub.drop(columns=["TEMP_ABS", "binning"], inplace=True)
    
    # グローバル更新
    globals()["results_model"] = results_model
    globals()["results_OOT"] = results_OOT
    globals()["Sub"] = Sub

# --- 変数ごとの変換処理 ---
for var in ["ELO", "ELO_ALL", "SRS", "SRS_rank", "Win_perc", "PTS_for", "SOS", "SOS_final", "SeedNum",
            "CONF_ELO", "Wins_quant1", "win_perc_quant1", "even_quant1",
            "Wins_quant2", "win_perc_quant2", "even_quant2",
            "Wins_Neutral", "even_neutral", "Wins_Away", "win_perc_away", "even_away",
            "win_perc_nothome", "even_nothome"]:
    variable_transform(var)


In [ ]:
import numpy as np
import pandas as pd

# --- 1. 外れ値の除去 (remove_outlier) ---
def remove_outlier(var):
    # 学習データ results_model を基準に下限・上限を計算（1.5×IQR）
    q1 = results_model[var].quantile(0.25)
    q3 = results_model[var].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - iqr * 1.5
    upper = q3 + iqr * 1.5

    # 各データセットで対象変数をクリップし、"_tt" 列として保存
    results_model[var + "_tt"] = results_model[var].clip(lower, upper)
    results_OOT[var + "_tt"] = results_OOT[var].clip(lower, upper)
    Sub[var + "_tt"] = Sub[var].clip(lower, upper)
    
    # グローバル更新（必要に応じて）
    globals()["results_model"] = results_model
    globals()["results_OOT"] = results_OOT
    globals()["Sub"] = Sub

# --- 2. 標準化 (norm_adjust) ---
def norm_adjust(var):
    # 学習データの _tt 列の平均・標準偏差を算出
    mean_val = results_model[var + "_tt"].mean()
    std_val = results_model[var + "_tt"].std()
    # 各データセットで標準化
    results_model[var + "_norm"] = (results_model[var + "_tt"] - mean_val) / std_val
    results_OOT[var + "_norm"] = (results_OOT[var + "_tt"] - mean_val) / std_val
    Sub[var + "_norm"] = (Sub[var + "_tt"] - mean_val) / std_val
    
    globals()["results_model"] = results_model
    globals()["results_OOT"] = results_OOT
    globals()["Sub"] = Sub

# --- 3. WoE変数のバイナリ指標作成 (WoE_vars) ---
def WoE_vars(var):
    # results_model[var+"_WoE"] が既に算出されている前提
    # 絶対値の頻度テーブルを作成
    temp_table = results_model[var + "_WoE"].abs().value_counts().reset_index()
    temp_table.columns = ["WoE_val", "count"]
    temp_table.sort_values("WoE_val", inplace=True)
    temp_table.reset_index(drop=True, inplace=True)
    
    print(var)
    print(temp_table)
    
    # 各頻度ごとにバイナリ指標の列を作成
    for i, row in temp_table.iterrows():
        val = row["WoE_val"]
        colname = f"{var}_WoE_B{i+1}"
        results_model[colname] = np.where(results_model[var + "_WoE"] == val, 1,
                                    np.where(results_model[var + "_WoE"] * -1 == val, -1, 0))
        results_OOT[colname] = np.where(results_OOT[var + "_WoE"] == val, 1,
                                np.where(results_OOT[var + "_WoE"] * -1 == val, -1, 0))
        Sub[colname] = np.where(Sub[var + "_WoE"] == val, 1,
                          np.where(Sub[var + "_WoE"] * -1 == val, -1, 0))
    globals()["results_model"] = results_model
    globals()["results_OOT"] = results_OOT
    globals()["Sub"] = Sub

# --- 4. 各変数に対する処理呼び出し ---

# 例：外れ値除去と標準化
for var in ["ELO", "ELO_ALL", "SRS", "SRS_rank", "Win_perc", "PTS_for", "SOS", "SOS_final", "SeedNum", "CONF_ELO",
            "Wins_quant1", "win_perc_quant1", "even_quant1",
            "Wins_quant2", "win_perc_quant2", "even_quant2",
            "Wins_Neutral", "even_neutral", "Wins_Away", "win_perc_away", "even_away",
            "win_perc_nothome", "even_nothome"]:
    remove_outlier(var)
    norm_adjust(var)

# 例：WoE のバイナリ指標作成（注意：results_model などに既に var+"_WoE" 列がある前提）
for var in ["ELO", "ELO_ALL", "SRS", "SRS_rank", "Win_perc", "PTS_for", "SOS", "SOS_final", "SeedNum", "CONF_ELO",
            "Wins_quant1", "win_perc_quant1", "even_quant1",
            "Wins_quant2", "win_perc_quant2", "even_quant2",
            "Wins_Neutral", "even_neutral", "Wins_Away", "win_perc_away", "even_away",
            "win_perc_nothome", "even_nothome"]:
    WoE_vars(var)


In [ ]:
import pandas as pd
import statsmodels.api as sm

# 例: results_model はすでに学習用データの DataFrame として用意され、
# outcome 変数は "result"、候補変数は以下の predictors リストに含まれているとする

predictors = [
    "ELO_norm", "ELO_ALL_norm", "SRS_norm", "SRS_rank_norm", "Win_perc_norm",
    "PTS_for_norm", "SOS_norm", "SOS_final_norm", "SeedNum_norm", "CONF_ELO_norm",
    "Wins_quant1_norm", "win_perc_quant1_norm", "even_quant1_norm",
    "Wins_quant2_norm", "win_perc_quant2_norm", "even_quant2_norm",
    "Wins_Neutral_norm", "even_neutral_norm", "Wins_Away_norm", "win_perc_away_norm",
    "even_away_norm", "win_perc_nothome_norm", "even_nothome_norm",
    "SRS_WoE_B1", "SRS_WoE_B2", "SRS_WoE_B3", "SRS_WoE_B4",
    "ELO_WoE_B1", "ELO_WoE_B2", "ELO_WoE_B3", "ELO_WoE_B4", "ELO_WoE_B5",
    "ELO_ALL_WoE_B1", "ELO_ALL_WoE_B2", "ELO_ALL_WoE_B3", "ELO_ALL_WoE_B4", "ELO_ALL_WoE_B5",
    "SRS_rank_WoE_B1", "SRS_rank_WoE_B2", "SRS_rank_WoE_B3", "SRS_rank_WoE_B4", "SRS_rank_WoE_B5",
    "Win_perc_WoE_B1", "Win_perc_WoE_B2", "Win_perc_WoE_B3", "Win_perc_WoE_B4",
    "PTS_for_WoE_B1", "PTS_for_WoE_B2",
    "SOS_WoE_B1", "SOS_WoE_B2", "SOS_WoE_B3", "SOS_WoE_B4",
    "SOS_final_WoE_B1", "SOS_final_WoE_B2", "SOS_final_WoE_B3",
    "SeedNum_WoE_B1", "SeedNum_WoE_B2", "SeedNum_WoE_B3", "SeedNum_WoE_B4", "SeedNum_WoE_B5", "SeedNum_WoE_B6",
    "CONF_ELO_WoE_B1", "CONF_ELO_WoE_B2", "CONF_ELO_WoE_B3", "CONF_ELO_WoE_B4", "CONF_ELO_WoE_B5",
    "Wins_quant1_WoE_B1", "Wins_quant1_WoE_B2", "Wins_quant1_WoE_B3", "Wins_quant1_WoE_B4", "Wins_quant1_WoE_B5", "Wins_quant1_WoE_B6",
    "win_perc_quant1_WoE_B1", "win_perc_quant1_WoE_B2", "win_perc_quant1_WoE_B3", "win_perc_quant1_WoE_B4", "win_perc_quant1_WoE_B5",
    "even_quant1_WoE_B1", "even_quant1_WoE_B2", "even_quant1_WoE_B3", "even_quant1_WoE_B4",
    "Wins_quant2_WoE_B1", "Wins_quant2_WoE_B2", "Wins_quant2_WoE_B3", "Wins_quant2_WoE_B4",
    "win_perc_quant2_WoE_B1", "win_perc_quant2_WoE_B2", "win_perc_quant2_WoE_B3", "win_perc_quant2_WoE_B4",
    "even_quant2_WoE_B1", "even_quant2_WoE_B2", "even_quant2_WoE_B3", "even_quant2_WoE_B4",
    "Wins_Neutral_WoE_B1", "Wins_Neutral_WoE_B2", "Wins_Neutral_WoE_B3", "Wins_Neutral_WoE_B4",
    "even_neutral_WoE_B1", "even_neutral_WoE_B2", "even_neutral_WoE_B3",
    "Wins_Away_WoE_B1", "Wins_Away_WoE_B2", "Wins_Away_WoE_B3", "Wins_Away_WoE_B4",
    "win_perc_away_WoE_B1", "win_perc_away_WoE_B2",
    "even_away_WoE_B1", "even_away_WoE_B2", "even_away_WoE_B3",
    "win_perc_nothome_WoE_B1", "win_perc_nothome_WoE_B2", "win_perc_nothome_WoE_B3",
    "even_nothome_WoE_B1", "even_nothome_WoE_B2", "even_nothome_WoE_B3", "even_nothome_WoE_B4"
]

# --- ステップワイズ変数選択用関数 ---
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.05, threshold_out=0.10,
                       verbose=True):
    """
    前向き・後退選択によるステップワイズ変数選択
    X: 説明変数 DataFrame
    y: 目的変数 (Series)
    initial_list: 最初から入れる変数リスト（通常は空リスト）
    threshold_in: 追加の閾値
    threshold_out: 削除の閾値
    """
    included = list(initial_list)
    while True:
        changed = False
        # 前向き追加ステップ
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index=excluded, dtype=float)
        for new_col in excluded:
            model = sm.Logit(y, sm.add_constant(X[included + [new_col]])).fit(disp=0)
            new_pval[new_col] = model.pvalues[new_col]
        if not new_pval.empty:
            best_pval = new_pval.min()
            if best_pval < threshold_in:
                best_feature = new_pval.idxmin()
                included.append(best_feature)
                changed = True
                if verbose:
                    print(f"Add {best_feature:30} with p-value {best_pval:.6f}")
        # 後退削除ステップ
        model = sm.Logit(y, sm.add_constant(X[included])).fit(disp=0)
        # 定数を除いた p 値を確認
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max()
        if worst_pval > threshold_out:
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            changed = True
            if verbose:
                print(f"Drop {worst_feature:30} with p-value {worst_pval:.6f}")
        if not changed:
            break
    return included

# --- 目的変数と説明変数の準備 ---
y = results_model["result"]
X = results_model[predictors]  # predictors は上記リスト

# --- ステップワイズ選択の実行 ---
selected_features = stepwise_selection(X, y, initial_list=[], threshold_in=0.05, threshold_out=0.10, verbose=True)
print("Selected features:")
print(selected_features)

# --- 最終モデルのフィッティング ---
X_selected = sm.add_constant(X[selected_features])
final_model = sm.Logit(y, X_selected).fit(disp=0)
print(final_model.summary())

# 結果を model_tourn_M として保存
model_tourn_M = final_model


In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# === モデル評価 ===

# 1. 学習データに対して予測
results_model["pred"] = model_tourn.predict(sm.add_constant(results_model))
results_model["error"] = (results_model["result"] - results_model["pred"])**2
print("Train MSE:", results_model["error"].mean())

# 2. 検証データに対して予測
results_OOT["pred"] = model_tourn.predict(sm.add_constant(results_OOT))
results_OOT["error"] = (results_OOT["result"] - results_OOT["pred"])**2
print("OOT MSE:", results_OOT["error"].mean())

print("Train rows:", results_model.shape[0])
print("OOT rows:", results_OOT.shape[0])
print("Error by Season (OOT):")
print(results_OOT.groupby("Season")["error"].mean())

# 3. 検証データのコピー（結果保存用）
results_OOT_M = results_OOT.copy()

# 4. 提出用データ予測
Sub["pred"] = model_tourn.predict(sm.add_constant(Sub))

# === 以下、各外部ファイルの読み込みと merge 処理 ===

# 読み込み
WNCAATourneyCompactResults = pd.read_csv(os.apeh.join(INPUT_PATH,"WNCAATourneyCompactResults.csv", sep=","))
TeamConferences = pd.read_csv(os.apeh.join(INPUT_PATH,"WTeamConferences.csv", sep=","))
SampleSubmissionWarmup = pd.read_csv(os.apeh.join(INPUT_PATH,"SampleSubmission2023.csv", sep=","))
NCAATourneySeeds = pd.read_csv(os.apeh.join(INPUT_PATH,"/WNCAATourneySeeds.csv", sep=","))
ELO_Annual_M = pd.read_csv("ELO_Annual_M.csv", sep=",")
ELO_CONF = pd.read_csv("ELO_CONF_M.csv", sep=",")
ELO_Total = pd.read_csv("ELO_Total_M.csv", sep=",")
SRS_day = pd.read_csv("M_SRS_day.csv", sep=",")
final_stats = pd.read_csv("Final_Stats.csv", sep=",")
quant_combined = pd.read_csv("final_quant.csv", sep=",")

# --- Sub の前処理 ---
# ID から Season, Team1, Team2 を抽出
Sub["Season"] = Sub["ID"].str.slice(0, 4).astype(int)
Sub["Team1"] = Sub["ID"].str.slice(5, 9).astype(int)
Sub["Team2"] = Sub["ID"].str.slice(10, 14).astype(int)
Sub = Sub[Sub["Team1"] <= 3100].copy()

# --- ELO_Annual_M 結合 ---
Sub = Sub.merge(ELO_Annual_M[["TeamID", "Season", "ELO"]], how="left",
                left_on=["Team1", "Season"], right_on=["TeamID", "Season"])
Sub.rename(columns={"ELO": "ELO_T1"}, inplace=True)
Sub.drop(columns=["TeamID"], inplace=True)
Sub = Sub.merge(ELO_Annual_M[["TeamID", "Season", "ELO"]], how="left",
                left_on=["Team2", "Season"], right_on=["TeamID", "Season"])
Sub.rename(columns={"ELO": "ELO_T2"}, inplace=True)
Sub.drop(columns=["TeamID"], inplace=True)

# --- TeamConferences 結合 ---
Sub = Sub.merge(TeamConferences[["TeamID", "Season", "ConfAbbrev"]], how="left",
                left_on=["Team1", "Season"], right_on=["TeamID", "Season"])
Sub.rename(columns={"ConfAbbrev": "ConfAbbrev_T1"}, inplace=True)
Sub.drop(columns=["TeamID"], inplace=True)
Sub = Sub.merge(TeamConferences[["TeamID", "Season", "ConfAbbrev"]], how="left",
                left_on=["Team2", "Season"], right_on=["TeamID", "Season"])
Sub.rename(columns={"ConfAbbrev": "ConfAbbrev_T2"}, inplace=True)
Sub.drop(columns=["TeamID"], inplace=True)

# --- ELO_CONF 結合 ---
Sub = Sub.merge(ELO_CONF[["ConfAbbrev", "Season", "Last_ELO"]], how="left",
                left_on=["ConfAbbrev_T1", "Season"], right_on=["ConfAbbrev", "Season"])
Sub.rename(columns={"Last_ELO": "CONF_ELO_T1"}, inplace=True)
Sub.drop(columns=["ConfAbbrev"], inplace=True)
Sub = Sub.merge(ELO_CONF[["ConfAbbrev", "Season", "Last_ELO"]], how="left",
                left_on=["ConfAbbrev_T2", "Season"], right_on=["ConfAbbrev", "Season"])
Sub.rename(columns={"Last_ELO": "CONF_ELO_T2"}, inplace=True)
Sub.drop(columns=["ConfAbbrev"], inplace=True)

# --- ELO_Total 結合 ---
Sub = Sub.merge(ELO_Total[["Team", "Season", "Last_ELO"]], how="left",
                left_on=["Team1", "Season"], right_on=["Team", "Season"])
Sub.rename(columns={"Last_ELO": "ELO_ALL_T1"}, inplace=True)
Sub.drop(columns=["Team"], inplace=True)
Sub = Sub.merge(ELO_Total[["Team", "Season", "Last_ELO"]], how="left",
                left_on=["Team2", "Season"], right_on=["Team", "Season"])
Sub.rename(columns={"Last_ELO": "ELO_ALL_T2"}, inplace=True)
Sub.drop(columns=["Team"], inplace=True)

# --- SRS_day 結合 ---
srs_day = srs_day[srs_day["Team"] != "loc"].copy()
srs_day["TeamID"] = pd.to_numeric(srs_day["Team"])
srs_day = srs_day[["TeamID", "Season", "SRS", "SRS_rank"]]
Sub = Sub.merge(srs_day, how="left",
                left_on=["Team1", "Season"], right_on=["TeamID", "Season"])
Sub.rename(columns={"SRS": "SRS_T1", "SRS_rank": "SRS_rank_T1"}, inplace=True)
Sub.drop(columns=["TeamID"], inplace=True)
Sub = Sub.merge(srs_day, how="left",
                left_on=["Team2", "Season"], right_on=["TeamID", "Season"])
Sub.rename(columns={"SRS": "SRS_T2", "SRS_rank": "SRS_rank_T2"}, inplace=True)
Sub.drop(columns=["TeamID"], inplace=True)

# --- final_stats 結合 ---
Sub = Sub.merge(final_stats, how="left",
                left_on=["Team1", "Season"], right_on=["Team", "Season"])
Sub.rename(columns={"Win_perc": "Win_perc_T1", "PTS_for": "PTS_for_T1", "PTS_against": "PTS_against_T1",
                     "SOS": "SOS_T1", "SOS_final": "SOS_final_T1"}, inplace=True)
Sub.drop(columns=["Team"], inplace=True)
Sub = Sub.merge(final_stats, how="left",
                left_on=["Team2", "Season"], right_on=["Team", "Season"])
Sub.rename(columns={"Win_perc": "Win_perc_T2", "PTS_for": "PTS_for_T2", "PTS_against": "PTS_against_T2",
                     "SOS": "SOS_T2", "SOS_final": "SOS_final_T2"}, inplace=True)
Sub.drop(columns=["Team"], inplace=True)

# --- NCAATourneySeeds 結合 ---
Sub = Sub.merge(NCAATourneySeeds, how="left",
                left_on=["Team1", "Season"], right_on=["TeamID", "Season"])
Sub.rename(columns={"Seed": "Seed_T1"}, inplace=True)
Sub.drop(columns=["TeamID"], inplace=True)
Sub = Sub.merge(NCAATourneySeeds, how="left",
                left_on=["Team2", "Season"], right_on=["TeamID", "Season"])
Sub.rename(columns={"Seed": "Seed_T2"}, inplace=True)
Sub.drop(columns=["TeamID"], inplace=True)

# --- Seed 番号抽出 ---
Sub["SeedNum_T1"] = Sub["Seed_T1"].str.slice(1,3).astype(float)
Sub["SeedNum_T2"] = Sub["Seed_T2"].str.slice(1,3).astype(float)

# --- Quant 指標の結合 ---
# Team1側
Sub = Sub.merge(quant_combined, how="left",
                left_on=["Team1", "Season"], right_on=["Team", "Season"])
Sub.rename(columns={
    "Wins_quant1": "Wins_quant1_T1",
    "win_perc_quant1": "win_perc_quant1_T1",
    "even_quant1": "even_quant1_T1",
    "Wins_quant2": "Wins_quant2_T1",
    "win_perc_quant2": "win_perc_quant2_T1",
    "even_quant2": "even_quant2_T1",
    "Wins_Neutral": "Wins_Neutral_T1",
    "win_perc_neutral": "win_perc_neutral_T1",
    "even_neutral": "even_neutral_T1",
    "Wins_Away": "Wins_Away_T1",
    "win_perc_away": "win_perc_away_T1",
    "even_away": "even_away_T1",
    "win_perc_nothome": "win_perc_nothome_T1",
    "even_nothome": "even_nothome_T1"
}, inplace=True)
Sub.drop(columns=["Team"], inplace=True)
# Team2側
Sub = Sub.merge(quant_combined, how="left",
                left_on=["Team2", "Season"], right_on=["Team", "Season"])
Sub.rename(columns={
    "Wins_quant1": "Wins_quant1_T2",
    "win_perc_quant1": "win_perc_quant1_T2",
    "even_quant1": "even_quant1_T2",
    "Wins_quant2": "Wins_quant2_T2",
    "win_perc_quant2": "win_perc_quant2_T2",
    "even_quant2": "even_quant2_T2",
    "Wins_Neutral": "Wins_Neutral_T2",
    "win_perc_neutral": "win_perc_neutral_T2",
    "even_neutral": "even_neutral_T2",
    "Wins_Away": "Wins_Away_T2",
    "win_perc_away": "win_perc_away_T2",
    "even_away": "even_away_T2",
    "win_perc_nothome": "win_perc_nothome_T2",
    "even_nothome": "even_nothome_T2"
}, inplace=True)
Sub.drop(columns=["Team"], inplace=True)

# --- 差分計算 ---
Sub["ELO"] = Sub["ELO_T1"] - Sub["ELO_T2"]
Sub["CONF_ELO"] = Sub["CONF_ELO_T1"] - Sub["CONF_ELO_T2"]
Sub["ELO_ALL"] = Sub["ELO_ALL_T1"] - Sub["ELO_ALL_T2"]
Sub["SRS"] = Sub["SRS_T1"] - Sub["SRS_T2"]
Sub["SRS_rank"] = Sub["SRS_rank_T1"] - Sub["SRS_rank_T2"]
Sub["Win_perc"] = Sub["Win_perc_T1"] - Sub["Win_perc_T2"]
Sub["PTS_for"] = Sub["PTS_for_T1"] - Sub["PTS_for_T2"]
Sub["PTS_against"] = Sub["PTS_against_T1"] - Sub["PTS_against_T2"]
Sub["SOS"] = Sub["SOS_T1"] - Sub["SOS_T2"]
Sub["SOS_final"] = Sub["SOS_final_T1"] - Sub["SOS_final_T2"]
Sub["SeedNum"] = Sub["SeedNum_T1"] - Sub["SeedNum_T2"]

Sub["Wins_quant1"] = Sub["Wins_quant1_T1"] - Sub["Wins_quant1_T2"]
Sub["win_perc_quant1"] = Sub["win_perc_quant1_T1"] - Sub["win_perc_quant1_T2"]
Sub["even_quant1"] = Sub["even_quant1_T1"] - Sub["even_quant1_T2"]
Sub["Wins_quant2"] = Sub["Wins_quant2_T1"] - Sub["Wins_quant2_T2"]
Sub["win_perc_quant2"] = Sub["win_perc_quant2_T1"] - Sub["win_perc_quant2_T2"]
Sub["even_quant2"] = Sub["even_quant2_T1"] - Sub["even_quant2_T2"]
Sub["Wins_Neutral"] = Sub["Wins_Neutral_T1"] - Sub["Wins_Neutral_T2"]
Sub["win_perc_neutral"] = Sub["win_perc_neutral_T1"] - Sub["win_perc_neutral_T2"]
Sub["even_neutral"] = Sub["even_neutral_T1"] - Sub["even_neutral_T2"]
Sub["Wins_Away"] = Sub["Wins_Away_T1"] - Sub["Wins_Away_T2"]
Sub["win_perc_away"] = Sub["win_perc_away_T1"] - Sub["win_perc_away_T2"]
Sub["even_away"] = Sub["even_away_T1"] - Sub["even_away_T2"]
Sub["win_perc_nothome"] = Sub["win_perc_nothome_T1"] - Sub["win_perc_nothome_T2"]
Sub["even_nothome"] = Sub["even_nothome_T1"] - Sub["even_nothome_T2"]

# --- 14. 結果出力 ---
Sub.to_csv("Submission_Features.csv", index=False)
print(Sub.head())


In [ ]:
import pandas as pd
import numpy as np

# --- 1. データ読み込み ---
Sub = pd.read_csv(os.path.join(INPUT_PATH,"SampleSubmission2023.csv", sep=","))
elo_annual = pd.read_csv("ELO_Annual_M.csv", sep=",")
teamconf = pd.read_csv(os.path.join(INPUT_PATH,"MTeamConferences.csv", sep=",")
elo_conf = pd.read_csv("ELO_CONF_M.csv", sep=",")
elo_total = pd.read_csv("ELO_Total_M.csv", sep=",")
srs_day = pd.read_csv("M_SRS_day.csv", sep=",")
final_stats = pd.read_csv("Final_Stats.csv", sep=",")
ncaatourneyseeds = pd.read_csv(os.path.join(INPUT_PATH,"MNCAATourneySeeds.csv", sep=","))
quant_combined = pd.read_csv("final_quant.csv", sep=",")

# --- 2. ID から Season, Team1, Team2 の抽出 ---
Sub["Season"] = Sub["ID"].str.slice(0, 4).astype(int)
Sub["Team1"] = Sub["ID"].str.slice(5, 9).astype(int)
Sub["Team2"] = Sub["ID"].str.slice(10, 14).astype(int)

# --- 3. フィルタ：Team1 > 3100 のレコードのみ抽出 ---
Sub = Sub[Sub["Team1"] > 3100].copy()

# --- 4. ELO_Annual_M 結合 ---
Sub = Sub.merge(elo_annual[["TeamID", "Season", "ELO"]], how="left",
                left_on=["Team1", "Season"], right_on=["TeamID", "Season"])
Sub.rename(columns={"ELO": "ELO_T1"}, inplace=True)
Sub.drop(columns=["TeamID"], inplace=True)

Sub = Sub.merge(elo_annual[["TeamID", "Season", "ELO"]], how="left",
                left_on=["Team2", "Season"], right_on=["TeamID", "Season"])
Sub.rename(columns={"ELO": "ELO_T2"}, inplace=True)
Sub.drop(columns=["TeamID"], inplace=True)

# --- 5. TeamConferences 結合 ---
Sub = Sub.merge(teamconf[["TeamID", "Season", "ConfAbbrev"]], how="left",
                left_on=["Team1", "Season"], right_on=["TeamID", "Season"])
Sub.rename(columns={"ConfAbbrev": "ConfAbbrev_T1"}, inplace=True)
Sub.drop(columns=["TeamID"], inplace=True)

Sub = Sub.merge(teamconf[["TeamID", "Season", "ConfAbbrev"]], how="left",
                left_on=["Team2", "Season"], right_on=["TeamID", "Season"])
Sub.rename(columns={"ConfAbbrev": "ConfAbbrev_T2"}, inplace=True)
Sub.drop(columns=["TeamID"], inplace=True)

# --- 6. ELO_CONF 結合 ---
Sub = Sub.merge(elo_conf[["ConfAbbrev", "Season", "Last_ELO"]], how="left",
                left_on=["ConfAbbrev_T1", "Season"], right_on=["ConfAbbrev", "Season"])
Sub.rename(columns={"Last_ELO": "CONF_ELO_T1"}, inplace=True)
Sub.drop(columns=["ConfAbbrev"], inplace=True)

Sub = Sub.merge(elo_conf[["ConfAbbrev", "Season", "Last_ELO"]], how="left",
                left_on=["ConfAbbrev_T2", "Season"], right_on=["ConfAbbrev", "Season"])
Sub.rename(columns={"Last_ELO": "CONF_ELO_T2"}, inplace=True)
Sub.drop(columns=["ConfAbbrev"], inplace=True)

# --- 7. ELO_Total 結合 ---
Sub = Sub.merge(elo_total[["Team", "Season", "Last_ELO"]], how="left",
                left_on=["Team1", "Season"], right_on=["Team", "Season"])
Sub.rename(columns={"Last_ELO": "ELO_ALL_T1"}, inplace=True)
Sub.drop(columns=["Team"], inplace=True)

Sub = Sub.merge(elo_total[["Team", "Season", "Last_ELO"]], how="left",
                left_on=["Team2", "Season"], right_on=["Team", "Season"])
Sub.rename(columns={"Last_ELO": "ELO_ALL_T2"}, inplace=True)
Sub.drop(columns=["Team"], inplace=True)

# --- 8. SRS_day 結合 ---
srs_day = srs_day[srs_day["Team"] != "loc"].copy()
srs_day["TeamID"] = pd.to_numeric(srs_day["Team"])
srs_day = srs_day[["TeamID", "Season", "SRS", "SRS_rank"]]

Sub = Sub.merge(srs_day, how="left",
                left_on=["Team1", "Season"], right_on=["TeamID", "Season"])
Sub.rename(columns={"SRS": "SRS_T1", "SRS_rank": "SRS_rank_T1"}, inplace=True)
Sub.drop(columns=["TeamID"], inplace=True)

Sub = Sub.merge(srs_day, how="left",
                left_on=["Team2", "Season"], right_on=["TeamID", "Season"])
Sub.rename(columns={"SRS": "SRS_T2", "SRS_rank": "SRS_rank_T2"}, inplace=True)
Sub.drop(columns=["TeamID"], inplace=True)

# --- 9. final_stats 結合 ---
Sub = Sub.merge(final_stats, how="left",
                left_on=["Team1", "Season"], right_on=["Team", "Season"])
Sub.rename(columns={"Win_perc": "Win_perc_T1", "PTS_for": "PTS_for_T1", 
                     "PTS_against": "PTS_against_T1", "SOS": "SOS_T1", "SOS_final": "SOS_final_T1"}, inplace=True)
Sub.drop(columns=["Team"], inplace=True)

Sub = Sub.merge(final_stats, how="left",
                left_on=["Team2", "Season"], right_on=["Team", "Season"])
Sub.rename(columns={"Win_perc": "Win_perc_T2", "PTS_for": "PTS_for_T2", 
                     "PTS_against": "PTS_against_T2", "SOS": "SOS_T2", "SOS_final": "SOS_final_T2"}, inplace=True)
Sub.drop(columns=["Team"], inplace=True)

# --- 10. NCAATourneySeeds 結合 ---
Sub = Sub.merge(ncaatourneyseeds, how="left",
                left_on=["Team1", "Season"], right_on=["TeamID", "Season"])
Sub.rename(columns={"Seed": "Seed_T1"}, inplace=True)
Sub.drop(columns=["TeamID"], inplace=True)

Sub = Sub.merge(ncaatourneyseeds, how="left",
                left_on=["Team2", "Season"], right_on=["TeamID", "Season"])
Sub.rename(columns={"Seed": "Seed_T2"}, inplace=True)
Sub.drop(columns=["TeamID"], inplace=True)

# --- 11. Seed 番号の抽出 ---
Sub["SeedNum_T1"] = Sub["Seed_T1"].str.slice(1,3).astype(float)
Sub["SeedNum_T2"] = Sub["Seed_T2"].str.slice(1,3).astype(float)

# --- 12. 差分計算 ---
Sub["ELO"] = Sub["ELO_T1"] - Sub["ELO_T2"]
Sub["CONF_ELO"] = Sub["CONF_ELO_T1"]

In [ ]:
import pandas as pd
import numpy as np

# 例: results はすでに必要な列を持つ DataFrame とする

# 試合結果の設定：WTeamID < LTeamID の場合に1、そうでなければ0
results["result"] = np.where(results["WTeamID"] < results["LTeamID"], 1, 0)

# Score1: WTeamID < LTeamID の場合は WScore、それ以外は LScore
results["Score1"] = np.where(results["WTeamID"] < results["LTeamID"], results["WScore"], results["LScore"])
# Score2: WTeamID > LTeamID の場合は WScore、それ以外は LScore
results["Score2"] = np.where(results["WTeamID"] > results["LTeamID"], results["WScore"], results["LScore"])

# margin: OTがあれば0、そうでなければ Score1 - Score2
results["margin"] = np.where(results["NumOT"] > 0, 0, results["Score1"] - results["Score2"])

# Loc: WLoc が "H" なら 1、"A" なら -1、それ以外は 0
results["Loc"] = np.where(results["WLoc"] == "H", 1, np.where(results["WLoc"] == "A", -1, 0))

# 各種差分計算（Team1側の値から Team2側の値を引く）
results["ELO"]         = results["ELO_T1"]         - results["ELO_T2"]
results["CONF_ELO"]    = results["CONF_ELO_T1"]    - results["CONF_ELO_T2"]
results["ELO_ALL"]     = results["ELO_ALL_T1"]     - results["ELO_ALL_T2"]
results["SRS"]         = results["SRS_T1"]         - results["SRS_T2"]
results["SRS_rank"]    = results["SRS_rank_T1"]    - results["SRS_rank_T2"]
results["Win_perc"]    = results["Win_perc_T1"]    - results["Win_perc_T2"]
results["PTS_for"]     = results["PTS_for_T1"]     - results["PTS_for_T2"]
results["PTS_against"] = results["PTS_against_T1"] - results["PTS_against_T2"]
results["SOS"]         = results["SOS_T1"]         - results["SOS_T2"]
results["SOS_final"]   = results["SOS_final_T1"]   - results["SOS_final_T2"]
results["SeedNum"]     = results["SeedNum_T1"]     - results["SeedNum_T2"]

results["Wins_quant1"]     = results["Wins_quant1_T1"]     - results["Wins_quant1_T2"]
results["win_perc_quant1"] = results["win_perc_quant1_T1"] - results["win_perc_quant1_T2"]
results["even_quant1"]     = results["even_quant1_T1"]     - results["even_quant1_T2"]
results["Wins_quant2"]     = results["Wins_quant2_T1"]     - results["Wins_quant2_T2"]
results["win_perc_quant2"] = results["win_perc_quant2_T1"] - results["win_perc_quant2_T2"]
results["even_quant2"]     = results["even_quant2_T1"]     - results["even_quant2_T2"]

results["Wins_Neutral"]    = results["Wins_Neutral_T1"]    - results["Wins_Neutral_T2"]
results["win_perc_neutral"]= results["win_perc_neutral_T1"]- results["win_perc_neutral_T2"]
results["even_neutral"]    = results["even_neutral_T1"]    - results["even_neutral_T2"]

results["Wins_Away"]       = results["Wins_Away_T1"]       - results["Wins_Away_T2"]
results["win_perc_away"]   = results["win_perc_away_T1"]   - results["win_perc_away_T2"]
results["even_away"]       = results["even_away_T1"]       - results["even_away_T2"]

results["win_perc_nothome"]= results["win_perc_nothome_T1"]- results["win_perc_nothome_T2"]
results["even_nothome"]    = results["even_nothome_T1"]    - results["even_nothome_T2"]

# --- play-in 試合の除外 ---
# Rコードでは Seed_len1 != Seed_len2 となっているので、その条件に合致する行のみ抽出
results_temp = results[results["Seed_len1"] != results["Seed_len2"]].copy()

# --- 学習用データ（results_model）とOOT（results_OOT）の作成 ---
results_model = results_temp.copy()
# （例）results_model をさらに Season < 2017 に絞る場合は以下のように可能（ここではコメントアウト）
# results_model = results_temp[results_temp["Season"] < 2017].copy()
results_OOT = results_temp[results_temp["Season"] >= 2017].copy()

print("results_model 行数:", results_model.shape[0])
print("results_OOT 行数:", results_OOT.shape[0])


In [ ]:
import pandas as pd
import numpy as np

# --- 1. データ読み込み ---
Sub = pd.read_csv(os.path.join(INPUT_PATH,"SampleSubmission2023.csv", sep=","))
elo_annual = pd.read_csv("ELO_Annual_M.csv", sep=",")
teamconf = pd.read_csv(os.path.join(INPUT_PATH,"MTeamConferences.csv", sep=","))
elo_conf = pd.read_csv("ELO_CONF_M.csv", sep=",")
elo_total = pd.read_csv("ELO_Total_M.csv", sep=",")
srs_day = pd.read_csv("M_SRS_day.csv", sep=",")
final_stats = pd.read_csv("Final_Stats.csv", sep=",")
ncaatourneyseeds = pd.read_csv(os.path.join(INPUT_PATH,"MNCAATourneySeeds.csv", sep=","))
quant_combined = pd.read_csv("final_quant.csv", sep=",")

# --- 2. ID から Season, Team1, Team2 の抽出 ---
Sub["Season"] = Sub["ID"].str.slice(0, 4).astype(int)
Sub["Team1"] = Sub["ID"].str.slice(5, 9).astype(int)
Sub["Team2"] = Sub["ID"].str.slice(10, 14).astype(int)

# --- 3. フィルタ：Team1 > 3100 のレコードのみ抽出 ---
Sub = Sub[Sub["Team1"] > 3100].copy()

# --- 4. ELO_Annual_M の結合 ---
Sub = Sub.merge(elo_annual[["TeamID", "Season", "ELO"]],
                how="left", left_on=["Team1", "Season"], right_on=["TeamID", "Season"])
Sub.rename(columns={"ELO": "ELO_T1"}, inplace=True)
Sub.drop(columns=["TeamID"], inplace=True)

Sub = Sub.merge(elo_annual[["TeamID", "Season", "ELO"]],
                how="left", left_on=["Team2", "Season"], right_on=["TeamID", "Season"])
Sub.rename(columns={"ELO": "ELO_T2"}, inplace=True)
Sub.drop(columns=["TeamID"], inplace=True)

# --- 5. TeamConferences の結合 ---
Sub = Sub.merge(teamconf[["TeamID", "Season", "ConfAbbrev"]],
                how="left", left_on=["Team1", "Season"], right_on=["TeamID", "Season"])
Sub.rename(columns={"ConfAbbrev": "ConfAbbrev_T1"}, inplace=True)
Sub.drop(columns=["TeamID"], inplace=True)

Sub = Sub.merge(teamconf[["TeamID", "Season", "ConfAbbrev"]],
                how="left", left_on=["Team2", "Season"], right_on=["TeamID", "Season"])
Sub.rename(columns={"ConfAbbrev": "ConfAbbrev_T2"}, inplace=True)
Sub.drop(columns=["TeamID"], inplace=True)

# --- 6. ELO_CONF の結合 ---
Sub = Sub.merge(elo_conf[["ConfAbbrev", "Season", "Last_ELO"]],
                how="left", left_on=["ConfAbbrev_T1", "Season"], right_on=["ConfAbbrev", "Season"])
Sub.rename(columns={"Last_ELO": "CONF_ELO_T1"}, inplace=True)
Sub.drop(columns=["ConfAbbrev"], inplace=True)

Sub = Sub.merge(elo_conf[["ConfAbbrev", "Season", "Last_ELO"]],
                how="left", left_on=["ConfAbbrev_T2", "Season"], right_on=["ConfAbbrev", "Season"])
Sub.rename(columns={"Last_ELO": "CONF_ELO_T2"}, inplace=True)
Sub.drop(columns=["ConfAbbrev"], inplace=True)

# --- 7. ELO_Total の結合 ---
Sub = Sub.merge(elo_total[["Team", "Season", "Last_ELO"]],
                how="left", left_on=["Team1", "Season"], right_on=["Team", "Season"])
Sub.rename(columns={"Last_ELO": "ELO_ALL_T1"}, inplace=True)
Sub.drop(columns=["Team"], inplace=True)

Sub = Sub.merge(elo_total[["Team", "Season", "Last_ELO"]],
                how="left", left_on=["Team2", "Season"], right_on=["Team", "Season"])
Sub.rename(columns={"Last_ELO": "ELO_ALL_T2"}, inplace=True)
Sub.drop(columns=["Team"], inplace=True)

# --- 8. SRS_day の結合 ---
srs_day = srs_day[srs_day["Team"] != "loc"].copy()
srs_day["TeamID"] = pd.to_numeric(srs_day["Team"])
srs_day = srs_day[["TeamID", "Season", "SRS", "SRS_rank"]]

Sub = Sub.merge(srs_day, how="left",
                left_on=["Team1", "Season"], right_on=["TeamID", "Season"])
Sub.rename(columns={"SRS": "SRS_T1", "SRS_rank": "SRS_rank_T1"}, inplace=True)
Sub.drop(columns=["TeamID"], inplace=True)

Sub = Sub.merge(srs_day, how="left",
                left_on=["Team2", "Season"], right_on=["TeamID", "Season"])
Sub.rename(columns={"SRS": "SRS_T2", "SRS_rank": "SRS_rank_T2"}, inplace=True)
Sub.drop(columns=["TeamID"], inplace=True)

# --- 9. final_stats の結合 ---
Sub = Sub.merge(final_stats, how="left",
                left_on=["Team1", "Season"], right_on=["Team", "Season"])
Sub.rename(columns={"Win_perc": "Win_perc_T1", "PTS_for": "PTS_for_T1", 
                     "PTS_against": "PTS_against_T1", "SOS": "SOS_T1", "SOS_final": "SOS_final_T1"}, inplace=True)
Sub.drop(columns=["Team"], inplace=True)

Sub = Sub.merge(final_stats, how="left",
                left_on=["Team2", "Season"], right_on=["Team", "Season"])
Sub.rename(columns={"Win_perc": "Win_perc_T2", "PTS_for": "PTS_for_T2", 
                     "PTS_against": "PTS_against_T2", "SOS": "SOS_T2", "SOS_final": "SOS_final_T2"}, inplace=True)
Sub.drop(columns=["Team"], inplace=True)

# --- 10. NCAATourneySeeds の結合 ---
Sub = Sub.merge(ncaatourneyseeds, how="left",
                left_on=["Team1", "Season"], right_on=["TeamID", "Season"])
Sub.rename(columns={"Seed": "Seed_T1"}, inplace=True)
Sub.drop(columns=["TeamID"], inplace=True)

Sub = Sub.merge(ncaatourneyseeds, how="left",
                left_on=["Team2", "Season"], right_on=["TeamID", "Season"])
Sub.rename(columns={"Seed": "Seed_T2"}, inplace=True)
Sub.drop(columns=["TeamID"], inplace=True)

# --- 11. Seed 番号抽出 ---
Sub["SeedNum_T1"] = Sub["Seed_T1"].str.slice(1, 3).astype(float)
Sub["SeedNum_T2"] = Sub["Seed_T2"].str.slice(1, 3).astype(float)

# --- 12. 差分計算 ---
Sub["ELO"] = Sub["ELO_T1"] - Sub["ELO_T2"]
Sub["CONF_ELO"] = Sub["CONF_ELO_T1"] - Sub["CONF_ELO_T2"]
Sub["ELO_ALL"] = Sub["ELO_ALL_T1"] - Sub["ELO_ALL_T2"]
Sub["SRS"] = Sub["SRS_T1"] - Sub["SRS_T2"]
Sub["SRS_rank"] = Sub["SRS_rank_T1"] - Sub["SRS_rank_T2"]
Sub["Win_perc"] = Sub["Win_perc_T1"] - Sub["Win_perc_T2"]
Sub["PTS_for"] = Sub["PTS_for_T1"] - Sub["PTS_for_T2"]
Sub["PTS_against"] = Sub["PTS_against_T1"] - Sub["PTS_against_T2"]
Sub["SOS"] = Sub["SOS_T1"] - Sub["SOS_T2"]
Sub["SOS_final"] = Sub["SOS_final_T1"] - Sub["SOS_final_T2"]
Sub["SeedNum"] = Sub["SeedNum_T1"] - Sub["SeedNum_T2"]

# --- 13. Quant 指標の結合 ---
# Team1 側
Sub = Sub.merge(quant_combined, how="left",
                left_on=["Team1", "Season"], right_on=["Team", "Season"])
Sub.rename(columns={
    "Wins_quant1": "Wins_quant1_T1",
    "win_perc_quant1": "win_perc_quant1_T1",
    "even_quant1": "even_quant1_T1",
    "Wins_quant2": "Wins_quant2_T1",
    "win_perc_quant2": "win_perc_quant2_T1",
    "even_quant2": "even_quant2_T1",
    "Wins_Neutral": "Wins_Neutral_T1",
    "win_perc_neutral": "win_perc_neutral_T1",
    "even_neutral": "even_neutral_T1",
    "Wins_Away": "Wins_Away_T1",
    "win_perc_away": "win_perc_away_T1",
    "even_away": "even_away_T1",
    "win_perc_nothome": "win_perc_nothome_T1",
    "even_nothome": "even_nothome_T1"
}, inplace=True)
Sub.drop(columns=["Team"], inplace=True)

# Team2 側
Sub = Sub.merge(quant_combined, how="left",
                left_on=["Team2", "Season"], right_on=["Team", "Season"])
Sub.rename(columns={
    "Wins_quant1": "Wins_quant1_T2",
    "win_perc_quant1": "win_perc_quant1_T2",
    "even_quant1": "even_quant1_T2",
    "Wins_quant2": "Wins_quant2_T2",
    "win_perc_quant2": "win_perc_quant2_T2",
    "even_quant2": "even_quant2_T2",
    "Wins_Neutral": "Wins_Neutral_T2",
    "win_perc_neutral": "win_perc_neutral_T2",
    "even_neutral": "even_neutral_T2",
    "Wins_Away": "Wins_Away_T2",
    "win_perc_away": "win_perc_away_T2",
    "even_away": "even_away_T2",
    "win_perc_nothome": "win_perc_nothome_T2",
    "even_nothome": "even_nothome_T2"
}, inplace=True)
Sub.drop(columns=["Team"], inplace=True)

# --- 14. Quant 指標の差分計算 ---
Sub["Wins_quant1"] = Sub["Wins_quant1_T1"] - Sub["Wins_quant1_T2"]
Sub["win_perc_quant1"] = Sub["win_perc_quant1_T1"] - Sub["win_perc_quant1_T2"]
Sub["even_quant1"] = Sub["even_quant1_T1"] - Sub["even_quant1_T2"]

Sub["Wins_quant2"] = Sub["Wins_quant2_T1"] - Sub["Wins_quant2_T2"]
Sub["win_perc_quant2"] = Sub["win_perc_quant2_T1"] - Sub["win_perc_quant2_T2"]
Sub["even_quant2"] = Sub["even_quant2_T1"] - Sub["even_quant2_T2"]

Sub["Wins_Neutral"] = Sub["Wins_Neutral_T1"] - Sub["Wins_Neutral_T2"]
Sub["win_perc_neutral"] = Sub["win_perc_neutral_T1"] - Sub["win_perc_neutral_T2"]
Sub["even_neutral"] = Sub["even_neutral_T1"] - Sub["even_neutral_T2"]

Sub["Wins_Away"] = Sub["Wins_Away_T1"] - Sub["Wins_Away_T2"]
Sub["win_perc_away"] = Sub["win_perc_away_T1"] - Sub["win_perc_away_T2"]
Sub["even_away"] = Sub["even_away_T1"] - Sub["even_away_T2"]

Sub["win_perc_nothome"] = Sub["win_perc_nothome_T1"] - Sub["win_perc_nothome_T2"]
Sub["even_nothome"] = Sub["even_nothome_T1"] - Sub["even_nothome_T2"]

# --- 最終結果の確認 ---
print(Sub.head())


In [ ]:
import numpy as np
import pandas as pd
import scorecardpy as sc

def variable_transform(var):
    global results_model, results_OOT, Sub

    # --- 学習データ(results_model) の処理 ---
    # TEMP_ABS: 対象変数の絶対値
    results_model["TEMP_ABS"] = results_model[var].abs()
    # TEMP_result: 対象変数が非負なら result、負なら 1-result を設定
    results_model["TEMP_result"] = np.where(results_model[var] >= 0,
                                              results_model["result"],
                                              1 - results_model["result"])
    # scorecardpy で binning（woebin）を実行
    # ※ x: TEMP_ABS, y: TEMP_result を使用
    binning_dict = sc.woebin(results_model, y="TEMP_result", x=["TEMP_ABS"])
    # binning_dict は {"TEMP_ABS": <DataFrame>} となるので取得
    Bin_proposed = binning_dict["TEMP_ABS"]
    
    # IV の出力（scorecardpy の woebin 出力には total_iv 列がある場合）
    print(var)
    if "total_iv" in Bin_proposed.columns:
        print("IV =", Bin_proposed["total_iv"].iloc[0])
    else:
        print("IV not computed in Bin_proposed")
    
    # 学習データに対してビニング変数を生成
    # 生成される列名は "TEMP_ABS_binned"
    results_model = sc.woebin_ply(results_model, binning_dict, x=["TEMP_ABS"], var_skip=[], keep="all")
    binned_col = var + "_binned"
    # ここでは、生成された列 "TEMP_ABS_binned" をリネームして使用
    results_model.rename(columns={"TEMP_ABS_binned": binned_col}, inplace=True)
    # binning は binned_col の先頭2文字を数値に変換
    results_model["binning"] = results_model[binned_col].astype(str).str[:2].astype(int)
    
    # IVテーブル（woebin の結果）を merge 用に準備
    merge_temp = Bin_proposed.copy().reset_index(drop=True)
    merge_temp["cut"] = np.arange(1, len(merge_temp) + 1)
    # WoE 値の Inf 修正（Inf, -Inf の場合）
    merge_temp["WoE"] = merge_temp["WoE"].replace(np.inf,
                        np.log((merge_temp["CntGood"] / merge_temp["CntGood"].sum()) / (1 / merge_temp["CntBad"].sum())))
    merge_temp["WoE"] = merge_temp["WoE"].replace(-np.inf,
                        np.log((1 / merge_temp["CntGood"].sum()) / (merge_temp["CntBad"] / merge_temp["CntBad"].sum())))
    
    # merge_temp の cut, WoE を学習データにマージ
    results_model = results_model.merge(merge_temp[["cut", "WoE"]], left_on="binning", right_on="cut", how="left")
    # 最終的な WoE 変数: var+"_WoE"。元の値が非負なら WoE、負なら -WoE
    results_model[var + "_WoE"] = np.where(results_model[var] >= 0,
                                           results_model["WoE"],
                                           -1 * results_model["WoE"])
    results_model.drop(columns=["WoE", "cut"], inplace=True)
    
    # --- 検証データ(results_OOT) の処理 ---
    results_OOT["TEMP_ABS"] = results_OOT[var].abs()
    results_OOT = sc.woebin_ply(results_OOT, binning_dict, x=["TEMP_ABS"], var_skip=[], keep="all")
    # 生成列名も "TEMP_ABS_binned" → リネーム
    results_OOT.rename(columns={"TEMP_ABS_binned": binned_col}, inplace=True)
    results_OOT["binning"] = results_OOT[binned_col].astype(str).str[:2].astype(int)
    results_OOT = results_OOT.merge(merge_temp[["cut", "WoE"]], left_on="binning", right_on="cut", how="left")
    results_OOT[var + "_WoE"] = np.where(results_OOT[var] >= 0,
                                         results_OOT["WoE"],
                                         -1 * results_OOT["WoE"])
    results_OOT.drop(columns=["WoE", "cut"], inplace=True)
    
    # --- 提出用データ(Sub) の処理 ---
    Sub["TEMP_ABS"] = Sub[var].abs()
    # ここでは np.digitize を用いて区間（bin）を取得
    # Bin_proposed の "bands" 列が存在する前提
    if "bands" in Bin_proposed.columns:
        # np.digitize: 結果は整数の配列
        Sub[binned_col] = np.digitize(Sub["TEMP_ABS"], bins=Bin_proposed["bands"].values, right=False)
    else:
        # bands 列がない場合、代替処理（ここでは woebin_ply を利用）
        Sub[binned_col] = sc.woebin_ply(Sub, binning_dict, x=["TEMP_ABS"], var_skip=[], keep="all")[binned_col]
    Sub["binning"] = Sub[binned_col].astype(str).str[:2].astype(int)
    Sub = Sub.merge(merge_temp[["cut", "WoE"]], left_on="binning", right_on="cut", how="left")
    Sub[var + "_WoE"] = np.where(Sub[var] >= 0,
                                 Sub["WoE"],
                                 -1 * Sub["WoE"])
    Sub.drop(columns=["WoE", "cut"], inplace=True)
    
    # 不要な TEMP_ABS, binning 列は削除（任意）
    results_model.drop(columns=["TEMP_ABS", "TEMP_result", "binning"], inplace=True)
    results_OOT.drop(columns=["TEMP_ABS", "binning"], inplace=True)
    Sub.drop(columns=["TEMP_ABS", "binning"], inplace=True)
    
    # グローバル更新
    globals()["results_model"] = results_model
    globals()["results_OOT"] = results_OOT
    globals()["Sub"] = Sub

# --- 変数ごとの変換処理 ---
for var in ["ELO", "ELO_ALL", "SRS", "SRS_rank", "PTS_for", "PTS_against",
            "SOS", "SOS_final", "SeedNum", "CONF_ELO",
            "Wins_quant1", "win_perc_quant1", "even_quant1",
            "Wins_quant2", "win_perc_quant2", "even_quant2",
            "Wins_Neutral", "even_neutral", "Wins_Away", "win_perc_away",
            "even_away", "win_perc_nothome", "even_nothome"]:
    variable_transform(var)


In [ ]:
import numpy as np
import pandas as pd

def remove_outlier(var):
    global results_model, results_OOT, Sub
    # 学習データ results_model を基準に下限・上限を計算（1.5×IQR）
    q1 = results_model[var].quantile(0.25)
    q3 = results_model[var].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    
    # 各データセットの新規列として var+"_tt" にクリップ値を設定
    results_model[var + "_tt"] = results_model[var].clip(lower, upper)
    # 検証用は results_model の四分位数を基準にクリップ
    results_OOT[var + "_tt"] = results_OOT[var].clip(lower, upper)
    Sub[var + "_tt"] = Sub[var].clip(lower, upper)
    
    # グローバル更新
    globals()["results_model"] = results_model
    globals()["results_OOT"] = results_OOT
    globals()["Sub"] = Sub

def WoE_vars(var):
    global results_model, results_OOT, Sub
    # temp_table: 絶対値の頻度テーブルを作成
    temp_table = results_model[var + "_WoE"].abs().value_counts().reset_index()
    temp_table.columns = ["WoE_val", "count"]
    temp_table = temp_table.sort_values("WoE_val").reset_index(drop=True)
    
    print(var)
    print(temp_table)
    
    # 各ユニークな WoE 値ごとに、バイナリ指標列を作成
    for i, row in temp_table.iterrows():
        value = row["WoE_val"]
        colname = f"{var}_WoE_B{i+1}"
        # 学習データ
        results_model[colname] = np.where(results_model[var + "_WoE"] == value, 1,
                                   np.where(results_model[var + "_WoE"] == -value, -1, 0))
        # 検証用データ
        results_OOT[colname] = np.where(results_OOT[var + "_WoE"] == value, 1,
                                   np.where(results_OOT[var + "_WoE"] == -value, -1, 0))
        # 提出用データ
        Sub[colname] = np.where(Sub[var + "_WoE"] == value, 1,
                           np.where(Sub[var + "_WoE"] == -value, -1, 0))
    # グローバル更新
    globals()["results_model"] = results_model
    globals()["results_OOT"] = results_OOT
    globals()["Sub"] = Sub

# --- 呼び出し例 ---
# まず、各対象変数に対して remove_outlier を実行
for var in ["ELO", "ELO_ALL", "SRS", "SRS_rank", "PTS_for", "PTS_against",
            "SOS", "SOS_final", "SeedNum", "CONF_ELO",
            "Wins_quant1", "win_perc_quant1", "even_quant1",
            "Wins_quant2", "win_perc_quant2", "even_quant2",
            "Wins_Neutral", "even_neutral", "Wins_Away", "win_perc_away",
            "even_away", "win_perc_nothome", "even_nothome"]:
    remove_outlier(var)

# 次に、各対象変数について WoE_vars を実行
for var in ["ELO", "ELO_ALL", "SRS", "SRS_rank", "PTS_for", "PTS_against",
            "SOS", "SOS_final", "SeedNum", "CONF_ELO",
            "Wins_quant1", "win_perc_quant1", "even_quant1",
            "Wins_quant2", "win_perc_quant2", "even_quant2",
            "Wins_Neutral", "even_neutral", "Wins_Away", "win_perc_away",
            "even_away", "win_perc_nothome", "even_nothome"]:
    WoE_vars(var)


In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, threshold_out=0.05, 
                       verbose=True, k_penalty=2):
    """
    前向き・後退選択によるステップワイズ変数選択（AIC ベース）
    
    X: 説明変数 DataFrame（切片なし）
    y: 目的変数（Series）
    initial_list: 最初に含める変数（通常は空リスト）
    threshold_in: 追加する際の p値閾値（例として 0.01）
    threshold_out: 除去する際の p値閾値（例として 0.05）
    k_penalty: AIC のペナルティ定数（ここでは k=2）
    
    ※ ここでは単純な基準により追加・削除を行っています。
    """
    included = list(initial_list)
    while True:
        changed = False
        # 前向きステップ：追加候補
        excluded = list(set(X.columns) - set(included))
        best_aic = np.inf
        best_feature = None
        for new_feature in excluded:
            # 現在の候補に新しい変数を追加
            try:
                model = sm.Logit(y, X[included + [new_feature]]).fit(disp=0)
                aic = model.aic  # statsmodels の aic は 2*k*#params + deviance
                if aic < best_aic:
                    best_aic = aic
                    best_feature = new_feature
            except Exception as e:
                continue
        # 現在のモデルの AIC
        if len(included) > 0:
            model_current = sm.Logit(y, X[included]).fit(disp=0)
            current_aic = model_current.aic
        else:
            current_aic = np.inf
        # AIC の改善があれば追加
        if best_feature is not None and best_aic < current_aic:
            included.append(best_feature)
            changed = True
            if verbose:
                print(f"Add {best_feature}, AIC = {best_aic:.2f}")
        # 後退ステップ：削除候補
        model = sm.Logit(y, X[included]).fit(disp=0)
        # p 値の最大値を調べる（定数項はないので全変数対象）
        pvals = model.pvalues
        worst_pval = pvals.max()
        if worst_pval > threshold_out:
            worst_feature = pvals.idxmax()
            new_list = list(included)
            new_list.remove(worst_feature)
            model_new = sm.Logit(y, X[new_list]).fit(disp=0)
            if model_new.aic < model.aic:
                included.remove(worst_feature)
                changed = True
                if verbose:
                    print(f"Drop {worst_feature}, AIC = {model_new.aic:.2f}")
        if not changed:
            break
    return included

# --- 候補変数リスト ---
predictors = [
    "Loc",
    "ELO_tt", "ELO_ALL_tt", "SRS_tt", "SRS_rank_tt", "PTS_for_tt", "PTS_against_tt",
    "SOS_tt", "SOS_final_tt", "SeedNum_tt", "CONF_ELO_tt",
    "Wins_quant1_tt", "win_perc_quant1_tt", "even_quant1_tt",
    "Wins_quant2_tt", "win_perc_quant2_tt", "even_quant2_tt",
    "Wins_Neutral_tt", "even_neutral_tt", "Wins_Away_tt", "win_perc_away_tt", "even_away_tt",
    "win_perc_nothome_tt", "even_nothome_tt",
    "ELO_WoE_B1", "ELO_WoE_B2", "ELO_WoE_B3", "ELO_WoE_B4",
    "ELO_ALL_WoE_B1", "ELO_ALL_WoE_B2", "ELO_ALL_WoE_B3", "ELO_ALL_WoE_B4", "ELO_ALL_WoE_B5",
    "SRS_WoE_B1", "SRS_WoE_B2", "SRS_WoE_B3", "SRS_WoE_B4",
    "SRS_rank_WoE_B1", "SRS_rank_WoE_B2", "SRS_rank_WoE_B3", "SRS_rank_WoE_B4", "SRS_rank_WoE_B5",
    "Win_perc_WoE_B1", "Win_perc_WoE_B2", "Win_perc_WoE_B3", "Win_perc_WoE_B4",
    "PTS_for_WoE_B1", "PTS_for_WoE_B2", "PTS_for_WoE_B3",
    "PTS_against_WoE_B1", "PTS_against_WoE_B2",
    "SOS_WoE_B1", "SOS_WoE_B2", "SOS_WoE_B3", "SOS_WoE_B4",
    "SOS_final_WoE_B1", "SOS_final_WoE_B2", "SOS_final_WoE_B3", "SOS_final_WoE_B4",
    "SeedNum_WoE_B1", "SeedNum_WoE_B2", "SeedNum_WoE_B3", "SeedNum_WoE_B4", "SeedNum_WoE_B5", "SeedNum_WoE_B6",
    "CONF_ELO_WoE_B1", "CONF_ELO_WoE_B2", "CONF_ELO_WoE_B3", "CONF_ELO_WoE_B4", "CONF_ELO_WoE_B5",
    "Wins_quant1_WoE_B1", "Wins_quant1_WoE_B2", "Wins_quant1_WoE_B3",
    "win_perc_quant1_WoE_B1", "win_perc_quant1_WoE_B2", "win_perc_quant1_WoE_B3", "win_perc_quant1_WoE_B4", "win_perc_quant1_WoE_B5",
    "even_quant1_WoE_B1", "even_quant1_WoE_B2", "even_quant1_WoE_B3", "even_quant1_WoE_B4",
    "Wins_quant2_WoE_B1", "Wins_quant2_WoE_B2", "Wins_quant2_WoE_B3",
    "win_perc_quant2_WoE_B1", "win_perc_quant2_WoE_B2", "win_perc_quant2_WoE_B3", "win_perc_quant2_WoE_B4", "win_perc_quant2_WoE_B5",
    "even_quant2_WoE_B1", "even_quant2_WoE_B2", "even_quant2_WoE_B3", "even_quant2_WoE_B4",
    "Wins_Neutral_WoE_B1", "Wins_Neutral_WoE_B2", "Wins_Neutral_WoE_B3", "Wins_Neutral_WoE_B4",
    "even_neutral_WoE_B1", "even_neutral_WoE_B2", "even_neutral_WoE_B3",
    "Wins_Away_WoE_B1", "Wins_Away_WoE_B2", "Wins_Away_WoE_B3",
    "win_perc_away_WoE_B1", "win_perc_away_WoE_B2",
    "even_away_WoE_B1", "even_away_WoE_B2", "even_away_WoE_B3", "even_away_WoE_B4",
    "win_perc_nothome_WoE_B1", "win_perc_nothome_WoE_B2", "win_perc_nothome_WoE_B3",
    "even_nothome_WoE_B1", "even_nothome_WoE_B2", "even_nothome_WoE_B3", "even_nothome_WoE_B4", "even_nothome_WoE_B5"
]

# --- 目的変数 ---
y = results_model["result"]
# --- 説明変数 (X) ---
X = results_model[predictors]

# --- ステップワイズ選択 ---
def stepwise_selection_aic(X, y, initial_features=[], verbose=True):
    """AIC を用いたステップワイズ変数選択（前向き・後退）"""
    included = list(initial_features)
    best_aic = np.inf
    while True:
        changed = False
        # 前向き: 追加候補
        excluded = list(set(X.columns) - set(included))
        aic_with_candidates = {}
        for new_feature in excluded:
            try:
                model = sm.Logit(y, X[included + [new_feature]]).fit(disp=0)
                aic_with_candidates[new_feature] = model.aic
            except Exception as e:
                continue
        if aic_with_candidates:
            best_candidate = min(aic_with_candidates, key=aic_with_candidates.get)
            candidate_aic = aic_with_candidates[best_candidate]
            # 現在のモデルの AIC
            if included:
                current_model = sm.Logit(y, X[included]).fit(disp=0)
                current_aic = current_model.aic
            else:
                current_aic = np.inf
            if candidate_aic < current_aic:
                included.append(best_candidate)
                best_aic = candidate_aic
                changed = True
                if verbose:
                    print(f"Add {best_candidate:30} AIC = {candidate_aic:.2f}")
        # 後退: 削除候補
        if included:
            model = sm.Logit(y, X[included]).fit(disp=0)
            pvals = model.pvalues
            worst_feature = pvals.idxmax()
            worst_pval = pvals.max()
            # ここでは p 値が高い場合に削除を検討
            if worst_pval > 0.10:
                new_features = list(included)
                new_features.remove(worst_feature)
                model_new = sm.Logit(y, X[new_features]).fit(disp=0)
                if model_new.aic < model.aic:
                    included.remove(worst_feature)
                    changed = True
                    if verbose:
                        print(f"Drop {worst_feature:30} AIC = {model_new.aic:.2f}")
        if not changed:
            break
    return included

selected_vars = stepwise_selection_aic(X, y, initial_features=[], verbose=True)
print("Selected features:")
print(selected_vars)

# --- 最終モデルのフィッティング ---
# 切片なしモデル：statsmodels では定数を追加しなければ切片なし
X_selected = X[selected_vars]
final_model = sm.Logit(y, X_selected).fit(disp=0)
print(final_model.summary())

# R の場合、model_tourn_W <- model_tourn
model_tourn_W = final_model


In [ ]:
import pandas as pd
import statsmodels.api as sm

# ※ここでは、最終的な候補変数 selected_features を用いて予測する前提とします。
# もしモデル構築時に定数項を加えていない場合は、add_constant は不要です。

# --- 学習データでの予測とエラー計算 ---
results_model["pred"] = model_tourn.predict(sm.add_constant(results_model[selected_features]))
results_model["error"] = (results_model["result"] - results_model["pred"])**2
print("Train MSE:", results_model["error"].mean())

# --- 検証データ (OOT) での予測とエラー計算 ---
results_OOT["pred"] = model_tourn.predict(sm.add_constant(results_OOT[selected_features]))
results_OOT["error"] = (results_OOT["result"] - results_OOT["pred"])**2
print("OOT MSE:", results_OOT["error"].mean())

print("Train rows:", results_model.shape[0])
print("OOT rows:", results_OOT.shape[0])

# --- OOT のシーズン別平均エラーの算出 ---
oot_error_by_season = results_OOT.groupby("Season")["error"].mean()
print("OOT error by Season:")
print(oot_error_by_season)

# --- results_OOT_W として検証用結果をコピー ---
results_OOT_W = results_OOT.copy()

# --- 提出用データ (Sub) に対して予測 ---
Sub["pred"] = model_tourn.predict(sm.add_constant(Sub[selected_features]))
Sub_W = Sub.copy()

# --- 学習用と検証用（女性側）検証結果の統合 ---
# ここでは、以前に計算済みの results_OOT_M と results_OOT_W のうち、
# "Season", "DayNum", "Team1", "Team2", "pred", "error" 列のみを抽出し結合しています。
results_OOT_combined = pd.concat([
    results_OOT_M[["Season", "DayNum", "Team1", "Team2", "pred", "error"]],
    results_OOT_W[["Season", "DayNum", "Team1", "Team2", "pred", "error"]]
], ignore_index=True)

print("Combined OOT MSE:", results_OOT_combined["error"].mean())
print("Combined OOT error by Season:")
print(results_OOT_combined.groupby("Season")["error"].mean())

# --- 提出用データの作成 ---
# Sub_M および Sub_W にはそれぞれ男性・女性の提出用結果が格納されていると仮定します。
# 以下では、"ID" と "pred" のみ抽出し、欠損値は 0.5 に置換した後、統合して最終提出ファイルを作成します。
Sub_M2 = Sub_M[["ID", "pred"]].copy()
Sub_M2["pred"] = Sub_M2["pred"].fillna(0.5)

Sub_W2 = Sub_W[["ID", "pred"]].copy()
Sub_W2["pred"] = Sub_W2["pred"].fillna(0.5)

Sub_final = pd.concat([Sub_M2, Sub_W2], ignore_index=True)
Sub_final.to_csv("submission.csv", index=False)
